In [1]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\Promotion Slide Duplicate\Promotion Replacement Function.ipynb"

## Parameters

In [2]:
normalized = True
promo_type = True
display_share = False  # True if Available
feature_share = False

In [3]:
ManufOrTopC ="Top Companies"
BrandOrTopB = "Top Brands"
prodORitem="SKU"

client_manuf = ["Edgewell Personal Care"]
client_brands = ["Schick", "Equate", "Cremo"]

categories = ["Manual Shave Men"]
sectors = ["System","Disposables"]
segments = ["Razors", "Refills", "Disposables"]
subsegments= []
subcategories= []

decimals = 2
sign = "Before"
currency = '$'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '

customareas=''
national = False
areas = ["RETAILER"]
regions_RET  =["Bj's And Sam's","Walmart"]
channels_RET = []
market_RET = []

regions_CHAN = []
channels_CHAN = []
market_CHAN = []

regions_CUST = []
channels_CUST = []
market_CUST = []

data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending March  2025"
years = ['2023', '2024', '2025']

subcatg_parent = "Segment"
subcatg_parent_list = segments
 
percent = 1000000
percentstr="'000 000"
start_date = "2022-04-01"	
end_date = "2025-04-01"
prodORitem = "SKU"

promo_col = []
selectedBrands = client_brands 
marketList = regions_RET + channels_RET + market_RET + regions_CHAN + channels_CHAN + market_CHAN 
notInScope = []
OpenEditData=True
direct_parent = {"Sector":"Category",
                "Segment":"Sector",
                "SubSegment":"Segment", 
                "SubCategory":"Segment"}

In [4]:
marketList = regions_RET + channels_RET + market_RET + regions_CHAN + channels_CHAN + market_CHAN + regions_CUST + channels_CUST + market_CUST
categoryList=categories +sectors+segments+subsegments+subcategories
defaults = {
    'Manual Shave Men': 6.69,
    'Disposables': 7.98,
    "System" : 5.03,
    "Razors" : 2.80,
    "Refills" : 6.58
}
diff_market_value = {

}

def totalsize (lis,defaultdic,diffmarketdic=[]):

    max_total_size = {
    f"{category} | {market}": diff_market_value.get(market.upper(), {}).get(category, defaults[category])
    for market in lis
    for category in defaults
    }

    return max_total_size  

max_total_size=totalsize(marketList,defaults,diff_market_value)

custom_colors = [
    RGBColor(91, 159, 153),    # Darker teal
    RGBColor(131, 199, 193),   # Brighter medium teal
    RGBColor(168, 216, 212),   # Original light teal
    RGBColor(198, 236, 232),   # Very light teal
    RGBColor(111, 179, 173),
    RGBColor(121, 189, 183)
]
# custom_colors = [
#     RGBColor(111, 179, 173),  
#     RGBColor(121, 189, 183),  
#     RGBColor(168, 216, 212),  
#     RGBColor(178, 226, 222),
# ]
 

In [5]:
max_total_size

{"Manual Shave Men | Bj's And Sam's": 6.69,
 "Disposables | Bj's And Sam's": 7.98,
 "System | Bj's And Sam's": 5.03,
 "Razors | Bj's And Sam's": 2.8,
 "Refills | Bj's And Sam's": 6.58,
 'Manual Shave Men | Walmart': 6.69,
 'Disposables | Walmart': 7.98,
 'System | Walmart': 5.03,
 'Razors | Walmart': 2.8,
 'Refills | Walmart': 6.58}

In [6]:

Scope = {
    "Category": categories,
    "Sector": sectors,
    "Segment": segments,
    "Subsegment": subsegments,
    "Subcategory": subcategories
}
suffixes = ["Category", "Sector", "Segment",'SubSegment', 'SubCategory']

In [7]:
def DetectHeader(df):
  # df = df.replace( np.nan, 0)
  vals=df.values
  vals=list(map(lambda x : all([type(i)==str for i in x ]),vals))
  # print(vals)
  break_point=0
  for i,v in enumerate(vals):
    if v:
      break_point=i
      # print(break_point)
      break  
  df.columns=df.iloc[break_point]
  df=df.iloc[break_point+1:,:]
  return df

## Reading datasets

In [8]:
loaded_data = {}
datasets_path = os.getcwd()+"/Promotion Datasets NewEX/"
datasets = os.listdir(datasets_path)
for d in datasets:
    with open(datasets_path+d, 'rb') as handle:
        globals()[d.split('.')[0]] = pd.read_pickle(handle)

## Data Cleaning

In [9]:
def cleaningData(data):
    """
    Clean and preprocess data in a dictionary of DataFrames.

    Parameters:
    - data (dict): Dictionary containing DataFrames.

    Returns:
    - dict: Dictionary containing cleaned DataFrames.
    """
    cleaned_data = {}
    
    # Iterate over each key-value pair in the input dictionary
    for key in data:
      
        # df=DetectHeader(data[key])
        df =data[key].copy()
        # Set column names and skip the first row
        if  df.columns.isna().any():
            continue
        # Perform specific cleaning operations based on the DataFrame columns and key
        if df.shape[0] > 0 and not 'National' in key:
            if 'Top Brands' in df.columns and f'{prodORitem}' in df.columns:
                df['Top Brands'] = df['Top Brands'].fillna(method='ffill')
                df[f'{prodORitem}'].fillna('', inplace=True)
                df.fillna(0, inplace=True)
                df['Top Brands'] = df['Top Brands'].apply(lambda x: 'Grand Total' if 'Grand Total' in x else x.replace('Total', '').strip())
                df = df.reset_index(drop=True)
            
            elif 'Top Brands' in df.columns:
                df['Top Brands'] = df['Top Brands'].fillna(method='ffill')
                #df.fillna(0, inplace=True)
                if normalized:
                    df.loc[:,~ df.columns.isin(['VSOD IYA','Value Uplift Normalized IYA'])] = df.loc[:,~ df.columns.isin(['VSOD IYA','Value Uplift Normalized IYA'])].fillna(0)
                    df['Promo Value Uplift vs YA'] = np.where(df['Value Uplift Normalized IYA'].isna(), None, df['Value Uplift Normalized IYA'] - 1)
                else:
                    df.loc[:, df.columns.isin(['VSOD IYA','Value Uplift IYA'])] = df.loc[:, df.columns.isin(['VSOD IYA','Value Uplift IYA'])].fillna(0)
                    df['Promo Value Uplift vs YA'] = np.where(df['Value Uplift IYA'].isna(), None, df['Value Uplift IYA'] - 1)
               
                df['VSOD Evaluation vs YA'] = np.where(df['VSOD IYA'].isna(), None, df['VSOD IYA'] - 1)

                df['Top Brands'] = df['Top Brands'].apply(lambda x: 'Grand Total' if 'Grand Total' in x else x.replace('Total', '').strip())
                df = df[~df['Top Brands'].str.contains('Total', case=False)]
                df = df[df['Total Size'] == 0].reset_index(drop=True)

       
                
            elif 'End of Week' in df.columns and f'{prodORitem}' in df.columns:
                df[f'{prodORitem}'] = df[f'{prodORitem}'].fillna(method='ffill')
                if normalized:
                    df = df[(df['Value Uplift (v. base) Normalized'] >= 0)]
                else:
                    df = df[(df['Value Uplift (v. base)'] >= 0)]
                    df['End of Week'] = pd.to_datetime(df['End of Week'], errors='coerce')
                    df = df[df['End of Week'].dt.year.isin([2023, 2024])]

                # df = df[(df['End of Week'].str.contains('2023|2024')) & (df['End of Week'].notna())]
                # df['End of Week'] = pd.to_datetime(df['End of Week'])
                new_start = "2023-07-31"
                df = df[(df['End of Week'] >= new_start) & (df['End of Week'] <= end_date)]
                df = df[~df[f'{prodORitem}'].str.contains('Total', case=False)].reset_index(drop=True)
                df = df[df['Promo Sales'] > 1000]
                if normalized:
                    df = df.dropna(subset=['Value Uplift (v. base) Normalized'])
                    df =  df[df['Value Uplift (v. base) Normalized']<10]
                else:
                    df = df.dropna(subset=['Value Uplift (v. base)'])
                    df = df[df['Value Uplift (v. base']<10]
                df.fillna(0, inplace=True)
                df = df.reset_index(drop=True)
                
            elif 'End of Week' in df.columns:
                df['End of Week'] = df['End of Week'].astype(str)
                df = df[~df['End of Week'].str.contains('Total', case=False)].reset_index(drop=True)
                df['End of Week'] = pd.to_datetime(df['End of Week'])
                df['End of Week'] = df['End of Week'].dt.strftime("%d-%b-%y")
                df = df[(df['End of Week'].str.contains('-22|-23|-24|Jan-25')) & (df['End of Week'].notna())]
                df['End of Week'] = pd.to_datetime(df['End of Week'])
                print(df['End of Week'])
                df = df[(df['End of Week'] >= start_date) & (df['End of Week'] <= end_date)]
                # df = df.dropna()
                
            elif 'Grand Total' in df.columns:
                if 'Sector' == df.columns[1]:
                    df[direct_parent["Sector"]].fillna(method='ffill', inplace= True)
                    df['Sector'] = df['Sector'].replace(0, np.nan)
                    df['Sector'].fillna(method='ffill', inplace=True)
                    df['Sector'] = df.apply(lambda row: row[direct_parent["Sector"]] if 'Total' in row[direct_parent["Sector"]] and row[direct_parent["Sector"]] != categories[0] else row['Sector'], axis=1)

                elif 'Segment' == df.columns[1]:
                    df['Segment'] = df['Segment'].replace(0, np.nan)  
                    df[direct_parent["Segment"]].fillna(method='ffill', inplace= True)          
                    df['Segment'] = df.apply(lambda row: row[direct_parent["Segment"]] if 'Total' in row[direct_parent["Segment"]] and row[direct_parent["Segment"]] != categories[0] else row['Segment'], axis=1)
                    df['Segment'].fillna(method='ffill', inplace=True)
                elif 'SubSegment' == df.columns[1]:
                    df['SubSegment'] = df['SubSegment'].replace(0, np.nan)
                    df[direct_parent["SubSegment"]].fillna(method='ffill', inplace= True)          
                    df['SubSegment'] = df.apply(lambda row: row[direct_parent["SubSegment"]] if 'Total' in row[direct_parent["SubSegment"]] and row[direct_parent["SubSegment"]] != categories[0] else row['SubSegment'], axis=1)
                    df['SubSegment'].fillna(method='ffill', inplace=True)
                elif 'SubCategory' == df.columns[1]:
                    df['SubCategory'] = df['SubCategory'].replace(0, np.nan)
                    df[direct_parent["SubCategory"]].fillna(method='ffill', inplace= True)          
                    df['SubCategory'] = df.apply(lambda row: row[direct_parent["SubCategory"]] if 'Total' in row[direct_parent["SubCategory"]] and row[direct_parent["SubCategory"]] != categories[0] else row['SubCategory'], axis=1)
                    df['SubCategory'].fillna(method='ffill', inplace=True)
                df = df.reset_index(drop=True)
        df.fillna(0, inplace=True)
            # Check if the key matches specific categories and modify the key accordingly

        if df.shape[0] > 0:
            cleaned_data[key] = df

    return cleaned_data


In [10]:
def cleaningdata_with_grand_total(data):
    """
    Clean and preprocess data in a dictionary of DataFrames.

    Parameters:
    - data (dict): Dictionary containing DataFrames.

    Returns:
    - dict: Dictionary containing cleaned DataFrames.
    """
    cleaningdata_with_grand_total = {}
    
    # Iterate over each key-value pair in the input dictionary
    for key in data:
   
        # df=DetectHeader(data[key])
        df =data[key].copy()

        if df.shape[0] > 0 and not 'National' in key:
            if 'Top Brands' in df.columns and f'{prodORitem}' in df.columns:
                df['Top Brands'] = df['Top Brands'].fillna(method='ffill')
                df[f'{prodORitem}'].fillna('', inplace=True)
                df.fillna(0, inplace=True)
                df['Top Brands'] = df['Top Brands'].apply(lambda x: 'Grand Total' if 'Grand Total' in x else x.replace('Total', '').strip())
            
            elif 'Top Brands' in df.columns:
                df['Top Brands'] = df['Top Brands'].fillna(method='ffill')
                if normalized:
                    df.loc[:,~ df.columns.isin(['VSOD IYA','Value Uplift Normalized IYA'])] = df.loc[:,~ df.columns.isin(['VSOD IYA','Value Uplift Normalized IYA'])].fillna(0)
                    df['Promo Value Uplift vs YA'] = np.where(df['Value Uplift Normalized IYA'].isna(), None, df['Value Uplift Normalized IYA'] - 1)
                else:
                    df.loc[:, df.columns.isin(['VSOD IYA','Value Uplift IYA'])] = df.loc[:, df.columns.isin(['VSOD IYA','Value Uplift IYA'])].fillna(0)
                    df['Promo Value Uplift vs YA'] = np.where(df['Value Uplift IYA'].isna(), None, df['Value Uplift IYA'] - 1)
               
                df['VSOD Evaluation vs YA'] = np.where(df['VSOD IYA'].isna(), None, df['VSOD IYA'] - 1)
                df['Top Brands'] = df['Top Brands'].apply(lambda x: 'Grand Total' if 'Grand Total' in x else x.replace('Total', '').strip())
                #df = df[~df['Top Brands'].str.contains('Total', case=False)]
                df = df[df['Total Size'] == 0].reset_index(drop=True)

            elif 'End of Week' in df.columns and f'{prodORitem}' in df.columns:
                df[f'{prodORitem}'] = df[f'{prodORitem}'].fillna(method='ffill')
                df = df[(df['End of Week'].str.contains('2023|2024')) & (df['End of Week'].notna())]
                df['End of Week'] = pd.to_datetime(df['End of Week'])
                df = df[(df['End of Week'] >= start_date) & (df['End of Week'] <= end_date)]
                df = df[~df[f'{prodORitem}'].str.contains('Total', case=False)].reset_index(drop=True)
                
                df = df[df['Promo Sales'] > 1000]
                if normalized:
                    df = df.dropna(subset=['Value Uplift (v. base) Normalized'])
                else:
                    df = df.dropna(subset=['Value Uplift (v. base)'])
                df.fillna(0, inplace=True)
                df = df.reset_index(drop=True)
                
            elif 'End of Week' in df.columns:
                df['End of Week'] = df['End of Week'].astype(str)
                df = df[~df['End of Week'].str.contains('Total', case=False)].reset_index(drop=True)
                df['End of Week'] = pd.to_datetime(df['End of Week'])
                df['End of Week'] = df['End of Week'].dt.strftime("%d-%b-%y")
                df = df[(df['End of Week'].str.contains('-21|-22|-23|Jan-24')) & (df['End of Week'].notna())]
                df['End of Week'] = pd.to_datetime(df['End of Week'])
                df = df[(df['End of Week'] >= start_date) & (df['End of Week'] <= end_date)]
                df = df.dropna()
                
            elif 'Grand Total' in df.columns:
                df['Sector'].fillna(method='ffill', inplace=True)
                df.fillna(0, inplace=True)
            
            # Check if the key matches specific categories and modify the key accordingly
            if key.split(' | ')[0] in categories and len(key.split(' | ')) == 3:
                modified_key = key.split(' | ')[1] + ' | ' + key.split(' | ')[2] + ' | ' + key.split(' | ')[0]
                if df.shape[0] > 0:
                    cleaningdata_with_grand_total[modified_key] = df
            else:
                if df.shape[0] > 0:
                    cleaningdata_with_grand_total[key] = df
    
    return cleaningdata_with_grand_total


In [11]:
def VSOD_Clean(VSOD_Data):
    """
    Clean and preprocess VSOD data in a dictionary of DataFrames.

    Parameters:
    - VSOD_Data (dict): Dictionary containing VSOD DataFrames.

    Returns:
    - dict: Dictionary containing cleaned VSOD DataFrames.
    """
    VSOD_cleaned = {}
    for key in VSOD_Data:
        # df=DetectHeader(VSOD_Data[key])
        df =VSOD_Data[key].copy()

        # print(key)
        if 'Sector' == df.columns[1]:
            df[direct_parent["Sector"]] = df[direct_parent["Sector"]].replace(0, np.nan)
            df[direct_parent["Sector"]].fillna(method='ffill', inplace = True)
            df['Sector'] = df['Sector'].replace(0, np.nan)
            df['Sector'] = df.apply(lambda row: row[direct_parent["Sector"]] if 'Total' in row[direct_parent["Sector"]] and row[direct_parent["Sector"]] != categories[0] else row['Sector'], axis=1)
            df = df[~(df[direct_parent["Sector"]].str.contains(r'\btotal\b', case=False) & 
                    (df[direct_parent["Sector"]] != categories[0])) | 
                    df[direct_parent["Sector"]].str.contains(r'\bGrand\b', case=False)].reset_index(drop=True)
            df['Sector'].fillna(method='ffill', inplace=True)
        elif 'Segment' == df.columns[1]:
            df[direct_parent["Segment"]].fillna(method='ffill', inplace=True)
            df['Segment'] = df['Segment'].replace(0, np.nan)
            # df=df[~df['Sector'].str.contains(r'\btotal\b', case=False) | df['Sector'].str.contains(r'\bGrand\b', case=False)].reset_index(drop=True)
            df['Segment'] = df.apply(lambda row: row[direct_parent["Segment"]] if 'Total' in row[direct_parent["Segment"]] and row[direct_parent["Segment"]] != categories[0] else row['Segment'], axis=1)
            df = df[~(df[direct_parent["Segment"]].str.contains(r'\btotal\b', case=False) & 
                    (df[direct_parent["Segment"]] != categories[0])) | 
                    df[direct_parent["Segment"]].str.contains(r'\bGrand\b', case=False)].reset_index(drop=True)
            df['Segment'].fillna(method='ffill', inplace=True)    
        elif 'SubSegment' == df.columns[1]:
            df['SubSegment'] = df['SubSegment'].replace(0, np.nan)
            df[direct_parent["SubSegment"]].fillna(method='ffill', inplace=True)
            # df=df[~df['Segment'].str.contains(r'\btotal\b', case=False) | df['Segment'].str.contains(r'\bGrand\b', case=False)].reset_index(drop=True)
            df['SubSegment'] = df.apply(lambda row: row[direct_parent["SubSegment"]] if 'Total' in row[direct_parent["SubSegment"]] and row[direct_parent["SubSegment"]] != categories[0] else row['SubSegment'], axis=1)
            df = df[~(df[direct_parent["SubSegment"]].str.contains(r'\btotal\b', case=False) & 
                    (df[direct_parent["SubSegment"]] != categories[0])) | 
                    df[direct_parent["SubSegment"]].str.contains(r'\bGrand\b', case=False)].reset_index(drop=True)
            df['SubSegment'].fillna(method='ffill', inplace=True)
        elif 'SubCategory' == df.columns[1]:
            df['SubCategory'] = df['SubCategory'].replace(0, np.nan)
            df[direct_parent["SubCategory"]].fillna(method='ffill', inplace=True)
            # df=df[~df['Segment'].str.contains(r'\btotal\b', case=False) | df['Segment'].str.contains(r'\bGrand\b', case=False)].reset_index(drop=True)
            df['SubCategory'] = df.apply(lambda row: row[direct_parent["SubCategory"]] if 'Total' in row[direct_parent["SubCategory"]] and row[direct_parent["SubCategory"]] != categories[0] else row['SubCategory'], axis=1)
            df = df[~(df[direct_parent["SubCategory"]].str.contains(r'\btotal\b', case=False) & 
                    (df[direct_parent["SubCategory"]] != categories[0])) | 
                    df[direct_parent["SubCategory"]].str.contains(r'\bGrand\b', case=False)].reset_index(drop=True)
            df['SubCategory'].fillna(method='ffill', inplace=True)
        df.fillna(0, inplace=True)
        df = df.reset_index(drop=True)
        if df.shape[0] > 0:
            VSOD_cleaned[key] = df
    return VSOD_cleaned


In [12]:
def merging(VSODClient_Cleaned, VSODCleaned, col):
    """
    Merge two dictionaries of DataFrames based on a common column.

    Parameters:
    - VSODClient_Cleaned (dict): Dictionary containing cleaned VSOD client DataFrames.
    - VSODCleaned (dict): Dictionary containing cleaned VSOD DataFrames.

    Returns:
    - dict: Dictionary containing merged DataFrames.
    """
    merged_dict = {}
    for key in VSODClient_Cleaned:
        # Merge DataFrames based on 'Sector' column
        #merged_df = pd.merge(VSODCleaned[key], VSODClient_Cleaned[key], on=col, how='left')
        merged_df = pd.merge(VSODClient_Cleaned[key],VSODCleaned[key], on=col, how='left')
        #merged_df = merged_df.dropna(subset=['Grand Total'])
        merged_df['Grand Total'] = merged_df['Grand Total'].fillna(0)
        merged_df = merged_df.fillna(0)
        if merged_df.shape[0]>0:
            merged_dict[key] = merged_df     
    return merged_dict


In [13]:
#modified_promotionBrandsP12M_display = cleaningData(display_data)

In [14]:
modified_promotionBrandsP12M = cleaningData(promotions_brands_P12M)

In [15]:
modified_promotionBrandsP12M_total = cleaningdata_with_grand_total(promotions_brands_P12M)

In [16]:
modified_promotionProductsP12M = cleaningData(promotions_products_P12M)

In [17]:
modified_promotionProductsP12M_updated = {}
for key, df in modified_promotionProductsP12M.items():
    df = df.copy()
    df = df[df[f'{prodORitem}'] != '']
    df = df[df['Promo Sales'] >= 10000]
    df = df.sort_values(by='Promo Value', ascending=False).reset_index(drop=True)
    if not df.empty:
        modified_promotionProductsP12M_updated[key] = df
modified_promotionProductsP12M_volumeuplift = modified_promotionProductsP12M_updated

In [18]:
modified_promotionEndOfWeek = cleaningData(promotions_EndOfWeek)

0     2023-01-08
1     2023-01-15
2     2023-01-22
3     2023-01-29
4     2023-02-05
         ...    
103   2024-12-29
104   2025-01-05
105   2025-01-12
106   2025-01-19
107   2025-01-26
Name: End of Week, Length: 108, dtype: datetime64[ns]
0     2023-01-08
1     2023-01-15
2     2023-01-22
3     2023-01-29
4     2023-02-05
         ...    
103   2024-12-29
104   2025-01-05
105   2025-01-12
106   2025-01-19
107   2025-01-26
Name: End of Week, Length: 108, dtype: datetime64[ns]
0    2023-10-01
1    2023-10-08
2    2023-10-15
3    2023-10-22
4    2023-10-29
5    2023-11-05
6    2023-11-12
7    2023-11-19
8    2023-11-26
9    2023-12-03
10   2023-12-10
11   2023-12-17
12   2023-12-24
13   2023-12-31
14   2024-01-07
15   2024-01-14
16   2024-01-21
17   2024-01-28
18   2024-02-04
19   2024-02-11
20   2024-02-18
21   2024-02-25
22   2024-03-03
23   2024-03-10
24   2024-03-17
25   2024-03-24
26   2024-03-31
27   2024-04-07
28   2024-04-14
29   2024-04-21
30   2024-04-28
31   2024-05-05
32   2

In [19]:
modified_valueUplift = cleaningData(value_uplift)

In [20]:
def cleaning13New(data, size_fix=1):
    """
    Clean and process data for specific brands and regions.

    Parameters:
    - data (dict): Dictionary containing raw data.

    Returns:
    - dict: Dictionary containing cleaned and processed data.
    """
    data_cleaned = {}
    
    # Define maximum total size for each combination of product type and region
    
    for key, df in data.items():
        # Skip processing if the region is 'NATIONAL' or 'National'
        if 'NATIONAL' in areas or 'National' in key:
            continue
        
        new_data = []

        df=data[key].copy()

        df['Top Brands'].fillna(method='ffill', inplace=True)
        
        # Filter out rows where 'Top Brands' is 'Grand Total' or 'Other'
        df = df[(df['Top Brands'] != 'Grand Total') & (df['Top Brands'] != 'Other')]
        # Remove 'GR' suffix from 'Total Size' and convert it to integer
        df['Total Size'] = df['Total Size'].str.extract('(\d+)', expand=False)
        df.fillna('0',inplace=True)
        df['Total Size'] = df['Total Size'].astype(int)
        df = df[df['Value Share'] > 0.01]
        
        # Sort data by 'Value Share' in descending order
        df = df.sort_values(by='Value Share', ascending=False).reset_index(drop=True)
        for i, brand in enumerate(df['Top Brands'].unique()):
            # Determine the product key based on the first two elements of the key

            product_key = key.split('|')[0] + '|' + key.split('|')[1]
            max_size = max_total_size.get(product_key, None)            
            # Filter rows for the current brand and check if total size is within the maximum allowed size
            if max_size is not None:
                
                brand_df = df[(df['Top Brands'] == brand) & (df['Total Size'] <= max_size*size_fix)]
            else:
                brand_df = pd.DataFrame()
                
            # Calculate recruitment ratio if the brand has data and total size is greater than zero
            #brand_total = df[(df['Top Brands'] == brand + ' Total')]['Promo Value'].values
            brand_total = df[(df['Top Brands'].str.strip() == (brand + ' Total').strip())]['Promo Value'].values
            
            
            if not brand_df.empty and brand_total.size > 0 and brand_total[0] > 0:
                
                brand_sum = brand_df['Promo Value'].sum() / brand_total[0]
     
                new_data.append({'Top Brands': brand, 'Recruitment': brand_sum, 'Consumption': 1 - brand_sum, 'Value Share': df['Value Share'][i], 'SUM':brand_df['Promo Value'].sum()})
        
        # Create a new DataFrame with cleaned data
        new = pd.DataFrame(new_data)
        new.fillna(0, inplace=True)
        # Add cleaned data to the dictionary if it contains non-zero rows
        if new.shape[0] != 0:
            data_cleaned[key] = new
        
    return data_cleaned


In [21]:
newModifiedBrands = cleaning13New(promotions_brands_P12M,1)

In [22]:
def merging(VSODClient_Cleaned, VSODCleaned, col):
    """
    Merge two dictionaries of DataFrames based on a common column.

    Parameters:
    - VSODClient_Cleaned (dict): Dictionary containing cleaned VSOD client DataFrames.
    - VSODCleaned (dict): Dictionary containing cleaned VSOD DataFrames.

    Returns:
    - dict: Dictionary containing merged DataFrames.
    """
    merged_dict = {}
    for key in VSODClient_Cleaned:
        # Merge DataFrames based on 'Sector' column
        #merged_df = pd.merge(VSODCleaned[key], VSODClient_Cleaned[key], on=col, how='left')
        merged_df = pd.merge(VSODClient_Cleaned[key],VSODCleaned[key], on=col, how='left')
        #merged_df = merged_df.dropna(subset=['Grand Total'])
        merged_df['Grand Total'] = merged_df['Grand Total'].fillna(0)
        merged_df = merged_df.fillna(0)
        if merged_df.shape[0]>0:
            merged_dict[key] = merged_df     
    return merged_dict


In [23]:
def dict_to_pivot_general(data_dict, pivot_col, value_col, aggfunc='sum', fill_value=pd.NA):
    result = {}
    for k, df in data_dict.items():
        frtcol=df.columns[0]
        seccol=df.columns[1]
        id_cols=[frtcol,seccol]
        df_clean = df.dropna(subset=id_cols + [pivot_col, value_col], how='any')
        
        pivot_df = df_clean.pivot_table(
            index=id_cols,
            columns=pivot_col,
            values=value_col,
            aggfunc=aggfunc,
            fill_value=fill_value
        ).reset_index()

        if isinstance(pivot_df.columns, pd.MultiIndex):
            pivot_df.columns = [col if not isinstance(col, tuple) else col[-1] for col in pivot_df.columns]
        
        # Sum brand columns (all columns after id_cols) to create 'Grand Total'
        brand_cols = pivot_df.columns[len(id_cols):]
        pivot_df['Grand Total'] = pivot_df[brand_cols].sum(axis=1)

        result[k] = pivot_df

    return result


In [24]:
# Define parameters for each pivot call in a list of tuples:
pivot_params = [
    (Sector_client_VSOD, 'Top Brands', 'Sector_client_VSODNew'),
    (Sector_manuf_VSOD, 'Top Companies', 'Sector_manuf_VSODNew'),
    (Segment_client_VSOD, 'Top Brands', 'Segment_client_VSODNew'),
    (Segment_manuf_VSOD, 'Top Companies', 'Segment_manuf_VSODNew'),
    (SubSegment_client_VSOD, 'Top Brands', 'SubSegment_client_VSODNew'),
    (SubSegment_manuf_VSOD, 'Top Companies', 'SubSegment_manuf_VSODNew'),
    (SubCategory_client_VSOD, 'Top Brands', 'SubCategory_client_VSODNew'),
    (SubCategory_manuf_VSOD, 'Top Companies', 'SubCategory_manuf_VSODNew')
]

# Prepare a dictionary to store results:
pivot_results = {}

for data_dict, pivot_col, result_name in pivot_params:
    pivot_results[result_name] = dict_to_pivot_general(
        data_dict=data_dict,
        pivot_col=pivot_col,
        value_col='VSOD',
        aggfunc='sum',
        fill_value=pd.NA
    )

# If you want to have these as separate variables in your workspace, you can unpack like:
Sector_client_VSODNew = pivot_results['Sector_client_VSODNew']
Sector_manuf_VSODNew = pivot_results['Sector_manuf_VSODNew']
Segment_client_VSODNew = pivot_results['Segment_client_VSODNew']
Segment_manuf_VSODNew = pivot_results['Segment_manuf_VSODNew']
SubSegment_client_VSODNew = pivot_results['SubSegment_client_VSODNew']
SubSegment_manuf_VSODNew = pivot_results['SubSegment_manuf_VSODNew']
SubCategory_client_VSODNew = pivot_results['SubCategory_client_VSODNew']
SubCategory_manuf_VSODNew = pivot_results['SubCategory_manuf_VSODNew']


In [25]:
if len(sectors) >0:
    a = VSOD_Clean(Sector_VSOD)
    if len(Sector_client_VSODNew) >0:
        b = cleaningData(Sector_client_VSODNew)
        sect_vsod_merged = merging(b,a, col=[direct_parent["Sector"],'Sector'])
    else:
        sect_vsod_merged = a
    c = cleaningData(Sector_manuf_VSODNew)
    for key in sect_vsod_merged:
        merged_df = pd.merge(sect_vsod_merged[key], c[key], on=[direct_parent["Sector"],'Sector'], how='left')
        if merged_df.shape[0]>0:
            sect_vsod_merged[key] = merged_df    

if len(segments) >0:
    a = VSOD_Clean(Segment_VSOD)
    if len(Segment_client_VSODNew) > 0:
        b = cleaningData(Segment_client_VSODNew)
        seg_vsod_merged = merging(a,b, col=[direct_parent["Segment"],'Segment'])
    else:
        seg_vsod_merged = a
    
    c = cleaningData(Segment_manuf_VSODNew)
    for key in seg_vsod_merged:
        # Merge DataFrames based on 'Sector' column
        merged_df = pd.merge(seg_vsod_merged[key], c[key], on=[direct_parent["Segment"],'Segment'], how='left')
        merged_df = merged_df.fillna(0)
        if merged_df.shape[0]>0:
            seg_vsod_merged[key] = merged_df    

if len(subsegments) >0:
    a = VSOD_Clean(SubSegment_VSOD)
    if len(SubSegment_client_VSODNew) > 0 :
        b = cleaningData(SubSegment_client_VSODNew)
        subseg_vsod_merged = merging(a,b, col=[direct_parent["SubSegment"],'SubSegment'])
    else:
        subseg_vsod_merged = a
    c = cleaningData(SubSegment_manuf_VSOD)
    for key in subseg_vsod_merged:
        # Merge DataFrames based on 'Sector' column
        merged_df = pd.merge(subseg_vsod_merged[key], c[key], on=[direct_parent["SubSegment"],'SubSegment'], how='left')
        merged_df = merged_df.fillna(0)
        if merged_df.shape[0]>0:
            subseg_vsod_merged[key] = merged_df    

if len(subcategories) >0:
    a = VSOD_Clean(SubCategory_VSOD)
    if len(SubCategory_client_VSODNew) > 0 :
        b = cleaningData(SubCategory_client_VSODNew)
        subcat_vsod_merged = merging(a,b, col=[direct_parent["SubCategory"],'SubCategory'])
    else:
        subcat_vsod_merged = a
    c = cleaningData(SubCategory_manuf_VSODNew)
    for key in subcat_vsod_merged:
        # Merge DataFrames based on 'Sector' column
        merged_df = pd.merge(subcat_vsod_merged[key], c[key], on=[direct_parent["SubCategory"],'SubCategory'], how='left')
        merged_df = merged_df.fillna(0)
        if merged_df.shape[0]>0:
            subcat_vsod_merged[key] = merged_df  

In [26]:
def splitkeys(dic, lis, parent,clientlist):
    """
    Splits the keys of a dictionary into new keys based on unique values in a specified column.
    Parameters:
    dic (dict): The input dictionary with DataFrames as values.
    lis (list): A list of sector names to filter by (if needed).
    parent (str): The column name used for splitting (e.g., 'Sector').
    
    Returns:
    dict: A new dictionary with updated keys and filtered DataFrames.
    """
    splitvsod = {}
    for key in dic.keys():
        for key in dic.keys():
            # Split the key into parts and check if all parts are in the valid list
    
                # Get a copy of the current DataFrame
            s = dic[key].copy()        
            for value in s[parent].unique():
                if isinstance(value, str) and not value.endswith("Total"):
                    new_key = f"{key} | {value}"                
                    filtered_df = s[s[parent].isin([value, f"{value} Total"])]   
                    for cli in clientlist:
                        needed_col = [filtered_df.columns[0], filtered_df.columns[1], "VSOD", cli]
                        missing_cols = [col for col in needed_col if col not in filtered_df.columns]
                        
                        if not missing_cols:
                            filtered_dfnew = filtered_df[needed_col]
                            splitvsod[new_key + " | " + cli] = filtered_dfnew
                        else:
                            print(f"Skipping {cli}: missing columns {missing_cols}")

    keys_to_remove = [
        k for k in splitvsod.keys() 
        if k.split(" | ")[-2] not in lis
    ]
    for k in keys_to_remove:
        del splitvsod[k]
    return splitvsod


In [27]:
client=client_brands+client_manuf
if len(sectors)!=0:
    sect_vsod_merged=splitkeys(sect_vsod_merged,categories,parent=direct_parent['Sector'],clientlist=client)
if len(segments)!=0:
    seg_vsod_merged=splitkeys(seg_vsod_merged,sectors,parent=direct_parent['Segment'],clientlist=client)
if len(subsegments)!=0:
    subseg_vsod_merged=splitkeys(subseg_vsod_merged,segments,parent=direct_parent['SubSegment'],clientlist=client)
if len(subcategories)!=0:
    subcat_vsod_merged=splitkeys(subcat_vsod_merged,segments,parent=direct_parent['SubCategory'],clientlist=client)

Skipping Equate: missing columns ['Equate']
Skipping Equate: missing columns ['Equate']
Skipping Equate: missing columns ['Equate']
Skipping Equate: missing columns ['Equate']
Skipping Equate: missing columns ['Equate']
Skipping Equate: missing columns ['Equate']
Skipping Equate: missing columns ['Equate']
Skipping Equate: missing columns ['Equate']


In [28]:
if len(sectors):
    sect_vsod_count =0
    for key,df in sect_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                sect_vsod_count +=1
    sect_vsod_count = sect_vsod_count *len(categories)
 
if len(segments):
    seg_vsod_count =0
    for key,df in seg_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                seg_vsod_count +=1
    #seg_vsod_count = seg_vsod_count * len(sectors) 
    seg_vsod_count = seg_vsod_count           
 
if len(subsegments) >0:
    subseg_vsod_count =0
    for key,df in subseg_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                subseg_vsod_count +=1
    #subseg_vsod_count =subseg_vsod_count *len(segments)
    subseg_vsod_count = subseg_vsod_count
 
if len(subcategories) >0:
    subcat_vsod_count =0
    for key,df in subcat_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                subcat_vsod_count +=1
    #subcat_vsod_count = subcat_vsod_count * len(subsegments)
    subcat_vsod_count = subcat_vsod_count

 

In [29]:
promotionsBrandSortedTotalFinal={}
promotionsBrandSortedTotal=dfSort(modified_promotionBrandsP12M, client_brands, "Top Brands", num=8,salesCol='Promo Value')
for key,df in promotionsBrandSortedTotal.items():
     df_client = selectClientBrands(promotionsBrandSortedTotal[key],'Top Brands', 'Promo Value')
     number_of_brands_needed = max(6 - len(df_client),0)
     df = df[~df['Top Brands'].isin(client_brands)]
     df = df.sort_values(by='Promo Value', ascending=False).head(number_of_brands_needed)
     df = pd.concat([df, df_client], ignore_index=True)
     df = df.sort_values(by='Promo Value', ascending=False).reset_index(drop=True)
     df = df[~df['Top Brands'].str.contains('Others', case=False)]
     df = df[~df['Top Brands'].str.contains('Grand Total', case=False)]
     df = df[df['Value Share'] > 0.01]
        
     df['VSOD Evaluation vs YA'] = df['VSOD Evaluation vs YA'].astype(float)
     df['Promo Value Uplift vs YA'] = df['Promo Value Uplift vs YA'].astype(float)
     
     if df.shape[0] >0:
          promotionsBrandSortedTotalFinal[key] = df

In [30]:
promotionsBrandNOTSortedTotalFinal={}
promotionsBrandNOTSortedTotalFinal=dfSort(modified_promotionBrandsP12M, client_brands, "Top Brands", num=8,salesCol='Promo Value')
for key,df in modified_promotionBrandsP12M.items():
     df = df.sort_values(by='Promo Value', ascending=False).reset_index(drop=True)
     df = df[~df['Top Brands'].str.contains('Others', case=False)]
     df = df[~df['Top Brands'].str.contains('Grand Total', case=False)]
     df = df[df['Value Share'] > 0.01]
     df['VSOD Evaluation vs YA'] = df['VSOD Evaluation vs YA'].astype(float)
     df['Promo Value Uplift vs YA'] = df['Promo Value Uplift vs YA'].astype(float)
     if df.shape[0] >0:
          promotionsBrandNOTSortedTotalFinal[key] = df

In [31]:
selectedBrands_og = selectedBrands
selectedBrands= selectedBrands + ["Grand Total"]

In [32]:
promotionsBrandsSelected={key:modified_promotionBrandsP12M_total[key][modified_promotionBrandsP12M_total[key]['Top Brands'].isin(selectedBrands)].sort_values(by='Promo Value',ascending=False) for key in modified_promotionBrandsP12M_total.keys()   if all(cat != key.split(' | ')[0] for cat in categories)}

In [33]:
for key in promotionsBrandsSelected:
    # Identify the Grand Total row (adjust this if necessary to match your data)
    grand_total_row = promotionsBrandsSelected[key].loc[promotionsBrandsSelected[key]['Top Brands'] == 'Grand Total']
    # Remove the Grand Total row from the dataframe
    sorted_df = promotionsBrandsSelected[key].loc[promotionsBrandsSelected[key]['Top Brands'] != 'Grand Total']
    # Concatenate the Grand Total row to the top of the dataframe
    promotionsBrandsSelected[key] = pd.concat([grand_total_row, sorted_df], ignore_index=True)

In [34]:
selectedBrands = selectedBrands_og

In [35]:
# Not including client brands
promotionsNotBrandsSelected = {
    key: modified_promotionBrandsP12M_total[key][
        ~modified_promotionBrandsP12M_total[key]['Top Brands'].isin(selectedBrands)
    ].sort_values(by='Value Share', ascending=False)
    for key in modified_promotionBrandsP12M_total.keys()
    if all(cat != key.split(' | ')[0] for cat in categories)
}


In [36]:
def concatAttribute(dic, marketList):
    """
    This function takes a dictionary of DataFrames and a list of markets, and concatenates
    the DataFrames by adding a 'SOURCE' column to each DataFrame.

    Parameters:
    dic (dict): A dictionary where keys are strings in the format 'market | source', and
                values are DataFrames containing market data.
    marketList (list): A list of market names (strings).

    Returns:
    dict: A dictionary where keys are market names and values are concatenated DataFrames
          with an added 'SOURCE' column.
    """
    # Initialize a defaultdict to store the resulting DataFrames
    marketDic = defaultdict(list)
    
    # Iterate through the list of markets
    for market in marketList:
        # Iterate through the items in the dictionary
        for key, value in dic.items():
            # Check if the market name matches the key's market part
            if market == key.split(' | ')[1]:
                # Extract the source part from the key and assign it to the 'SOURCE' column
                value['SOURCE'] = list(set(key.split(' | ')) - set([market]))[0]
                value = value[value['Value Share'] >0.01]
                value = value[~value['Top Brands'].str.contains('Other')].reset_index(drop=True)
                # Only include rows where 'SOURCE' is not 'National'
                if (value['SOURCE'] != 'National').all():
                    marketDic[market].append(value)

        # Concatenate all DataFrames in the list for each market
        if len(marketDic[market]) != 0:
            marketDic[market] = pd.concat(marketDic[market])
    
    return marketDic

def fillingMissingBrands(dic):
    """
    This function fills in missing brands for each market and source combination in the
    provided dictionary of DataFrames.

    Parameters:
    dic (dict): A dictionary where keys are market names and values are DataFrames
                containing market data with 'Top Brands' and 'SOURCE' columns.

    Returns:
    dict: The input dictionary with missing brands filled in each DataFrame.
    """
    # Iterate through the dictionary items
    for key, value in dic.items():
        # Get the unique list of top brands in the DataFrame
        brandList = value['Top Brands'].unique().tolist()
        # Iterate through the unique sources in the DataFrame
        for source in value['SOURCE'].unique():
            # Check if the number of unique brands for the source is less than the total unique brands
            if value[value['SOURCE'] == source]['Top Brands'].nunique() != len(brandList):
                # Find the missing brands for the source
                missingBrand = list(set(brandList) - set(value[value['SOURCE'] == source]['Top Brands'].unique()))
                # Create a DataFrame for the missing brands with the current source
                missingBrand = pd.DataFrame({'Top Brands': missingBrand, 'SOURCE': source}).explode('Top Brands')
                # Concatenate the missing brands DataFrame with the original DataFrame
                value = pd.concat([value, missingBrand]).replace(np.nan, 0).reset_index(drop=True)
        # Update the dictionary with the filled DataFrame
        dic[key] = value
    
    return dic


In [37]:
promotionsBrandsSelected['Disposables | Bj\'s And Sam\'s']

,Top Brands,Total Size,Promo Value,VSOD,VSOD IYA,Value Share,Promo Share,Value Uplift (v. base) Normalized,Value Uplift Normalized IYA,Volume Uplift (v. Base) Normalized,Volume Uplift Normalized IYA,Promo Value Uplift vs YA,VSOD Evaluation vs YA
0,Grand Total,0,16505432.0,0.55,1.447,1.00000,1.000000,0.245532,1.404748,0.424946,1.367674,0.404748,0.447
1,Schick,0,6252872.0,0.58,1.381,0.37415,0.378837,0.231346,0.982016,0.517432,1.186822,-0.017984,0.381


In [38]:
promotionsBrandsWithMarket=concatAttribute(promotionsBrandsSelected,marketList)
promotionsBrandsWithMarket = fillingMissingBrands(promotionsBrandsWithMarket)
promotionsNotBrandsWithMarket=concatAttribute(promotionsNotBrandsSelected,marketList)
promotionsNotBrandsWithMarket = fillingMissingBrands(promotionsNotBrandsWithMarket)

In [39]:
def split_market(data, Scope):
    final = {}
    for k,df in data.items():
        for key, value in Scope.items():
            df_market = df[df['SOURCE'].isin(value)]
            df_market = df_market.reset_index(drop=True)
            if df_market.shape[0] >0:
                final[k + ' | ' + value[0]] = df_market
    return final        

In [40]:
newpromotionsBrandsWithMarket = split_market(promotionsBrandsWithMarket,Scope)
newpromotionsNotBrandsWithMarket = split_market(promotionsNotBrandsWithMarket,Scope)

In [41]:
def concatAttributeNew(sorted):
    """
    Concatenate DataFrames from a sorted dictionary based on exact matches of categories, sectors, segments,
    subsegments, and subcategories. Adds a 'SOURCE' column to each DataFrame indicating its market.

    Parameters:
    sorted (dict): Dictionary with keys like 'category | sector | segment | brand' and values as DataFrames.
    categories, sectors, segments, subsegments, subcategories (list): Lists of strings to match exactly.

    Returns:
    dict: Dictionary with matched group names as keys and concatenated DataFrames as values.
    """
    # Combine all lists and preserve order without duplicates
    lis = list(dict.fromkeys(categories + sectors + segments + subsegments + subcategories))

    marketDic = defaultdict(list)
    concatenatedDic = {}

    for i in lis:
        for key, df in sorted.items():
            parts = key.split(' | ')
            if i in parts:
                # Determine market label
        
                market_label = parts[1]  # category

                df = df.copy()  # Avoid modifying original
                df['SOURCE'] = market_label
                marketDic[i].append(df)

        if marketDic[i]:
            concatenatedDic[i] = pd.concat(marketDic[i], ignore_index=True)

    return concatenatedDic



In [42]:
concated = concatAttributeNew(modified_promotionBrandsP12M)

In [43]:
def filter_data(modified_promotionProductsP12M):
    cleaned_data = {}
    for key in modified_promotionProductsP12M:
        df = modified_promotionProductsP12M[key]
        df = df[df[f'{prodORitem}'] != '']
        df=df.sort_values(by=['Promo Share'], ascending=False)
        # Filter and sort the DataFrame
        df['cumulative promo share'] = df['Promo Share'].cumsum()
        df = df[df['Discount Depth (%)'] >= 0.05]
        df = df[df['VSOD'] >= 0.05]
        df = df[df['cumulative promo share'] <= 0.8]
        df = df.sort_values(by='Incr Value', ascending=False).reset_index(drop=True)
        df = df.head(50)
        df['index'] = str(df.index + 1)
        df = df.reset_index(drop=True)
        if df.shape[0] >0:
            cleaned_data[key] = df
        #else:
            #print(key)
    return cleaned_data
        

In [44]:
new_modified_promotionProductsP12M = filter_data(modified_promotionProductsP12M)

In [45]:
def filter_data_Top(modified_promotionProductsP12M):
    cleaned_data = {}
    for key in modified_promotionProductsP12M:
        combined_df = pd.DataFrame() 
        for client in client_brands:
            
            df = modified_promotionProductsP12M[key]
            # Filter the DataFrame for the current client brand
            df = df[df[f'{prodORitem}'] != '']
            df = df[df['Top Brands'] == client]
            df = df.sort_values(by='Top Brands')
            df['Promo Share'] = pd.to_numeric(df['Promo Share'], errors='coerce')
            df['cumulative promo share'] = df.groupby('Top Brands')['Promo Share'].cumsum()
            df = df[df['cumulative promo share'] <= 0.8]
            df = df[df['Discount Depth (%)'] >= 0.05]
            df = df[df['VSOD'] >= 0.05]
            if df.shape[0] >0:
                combined_df = pd.concat([combined_df, df])
                combined_df = combined_df.sort_values(by='Incr Value', ascending=False).head(20).reset_index(drop=True)
        if combined_df.shape[0] > 0:
            cleaned_data[key] = combined_df.reset_index(drop=True)  # Store the combined DataFrame for the current key
                
    return cleaned_data
        

In [46]:
def filter_data_Bot(modified_promotionProductsP12M):
    cleaned_data = {}
    for key in modified_promotionProductsP12M:
        combined_df = pd.DataFrame() 
        for client in client_brands:
            df = modified_promotionProductsP12M[key]
            df = df[df[f'{prodORitem}'] != '']
            df = df[df['Top Brands'] == client]
            df = df.sort_values(by='Top Brands')
            df['Promo Share'] = pd.to_numeric(df['Promo Share'], errors='coerce')
            df['cumulative promo share'] = df.groupby('Top Brands')['Promo Share'].cumsum()
            df = df[df['Discount Depth (%)'] >= 0.05]
            df = df[df['VSOD'] >= 0.05]
            if df.shape[0] >0:
                combined_df = pd.concat([combined_df, df])
                combined_df = combined_df.sort_values(by='Incr Value', ascending=False).tail(20).reset_index(drop=True)
                combined_df = combined_df.sort_values(by ='Incr Value', ascending= True).reset_index(drop=True)
        if combined_df.shape[0] > 0:
            cleaned_data[key] = combined_df.reset_index(drop=True)  # Store the combined DataFrame for the current key
    return cleaned_data
        

In [47]:
top20clientonly = filter_data_Top(modified_promotionProductsP12M)
bottom20clientonly = filter_data_Bot(modified_promotionProductsP12M)

In [48]:
def promotionsEndOfWeekCleaning(promotions_EndOfWeek, notInScope, col='Top Brands'):
    """
    Clean promotions end of week data.

    Parameters:
    promotions_EndOfWeek (dict): Dictionary containing promotions end of week data.
    notInScope (list): List of items not in scope to be excluded.
    col (str): Column name to check for filtering. Default is 'Top Brands'.

    Returns:
    dict: Dictionary containing cleaned promotions end of week data.
    """
    promotionsEndOfWeek = {}
    for key, value in promotions_EndOfWeek.items():
        df = value.copy()
        if df.shape[0] != 0:
            modified_key = key
            flag = False if any(element in modified_key for element in notInScope) else True
            if flag:
                promotionsEndOfWeek[modified_key] = df[df[col] != 'Grand Total'].reset_index(drop=True).replace(np.nan, 0)
        else:
            print(key, ' Is empty')
    
    return promotionsEndOfWeek


In [49]:
mod=cleaningData(promotions_EndOfWeek)
promotionsEndOfWeekCleaned=promotionsEndOfWeekCleaning(mod,notInScope,col='End of Week')


0     2023-01-08
1     2023-01-15
2     2023-01-22
3     2023-01-29
4     2023-02-05
         ...    
103   2024-12-29
104   2025-01-05
105   2025-01-12
106   2025-01-19
107   2025-01-26
Name: End of Week, Length: 108, dtype: datetime64[ns]
0     2023-01-08
1     2023-01-15
2     2023-01-22
3     2023-01-29
4     2023-02-05
         ...    
103   2024-12-29
104   2025-01-05
105   2025-01-12
106   2025-01-19
107   2025-01-26
Name: End of Week, Length: 108, dtype: datetime64[ns]
0    2023-10-01
1    2023-10-08
2    2023-10-15
3    2023-10-22
4    2023-10-29
5    2023-11-05
6    2023-11-12
7    2023-11-19
8    2023-11-26
9    2023-12-03
10   2023-12-10
11   2023-12-17
12   2023-12-24
13   2023-12-31
14   2024-01-07
15   2024-01-14
16   2024-01-21
17   2024-01-28
18   2024-02-04
19   2024-02-11
20   2024-02-18
21   2024-02-25
22   2024-03-03
23   2024-03-10
24   2024-03-17
25   2024-03-24
26   2024-03-31
27   2024-04-07
28   2024-04-14
29   2024-04-21
30   2024-04-28
31   2024-05-05
32   2

In [50]:

brandMarketCategory = [key for key in promotionsEndOfWeekCleaned.keys() if any(cat in key.split(' | ')[0] for cat in categories )]
if len(sectors) != 0:
    brandMarketSector = [key for key in promotionsEndOfWeekCleaned.keys() if any(cat == key.split(' | ')[0] for cat in sectors )]
if len(segments) != 0:
    brandMarketSegment = [key for key in promotionsEndOfWeekCleaned.keys() if any(cat == key.split(' | ')[0] for cat in segments )]
if len(subsegments) != 0:
    brandMarketSubSegment = [key for key in promotionsEndOfWeekCleaned.keys() if any(cat == key.split(' | ')[0] for cat in subsegments )]
if len(subcategories) != 0:
    brandMarketSubCategory = [key for key in promotionsEndOfWeekCleaned.keys() if any(cat == key.split(' | ')[0] for cat in subcategories )]


In [51]:
def completeDates(dfList, promotionsEndOfWeekCleaned):
    """
    Complete dates for each dataframe in dfList based on promotionsEndOfWeekCleaned.
    Parameters:
    dfList (list): List of dataframe keys.
    promotionsEndOfWeekCleaned (dict): Dictionary containing cleaned promotions end of week data.
    Returns:
    tuple: Tuple containing EndOfWeekcompletDate (dictionary), dfGroup (list), and dic (dictionary).
    """
    # Create a list of unique brand-category combinations
    brandCatList = sorted(set(key.split(' | ')[0] + ' | ' + key.split(' | ')[2] for key in dfList))
    EndOfWeekcompletDate = {}
    dfGroup = []
    dic = defaultdict(int)
    for key in brandCatList:
        for name in dfList:
            if (key.split(' | ')[0] == name.split(' | ')[0]) and (key.split(' | ')[1] == name.split(' | ')[2]):
                dic[key] += 1
                
    # Iterate over unique brand-category combinations
    for name in dic.keys():
        # Get dataframe keys associated with the current brand-category combination
        dfName = [key for key in dfList if name == (key.split(' | ')[0] + ' | ' + key.split(' | ')[2])]
        uniqueDates = pd.concat([promotionsEndOfWeekCleaned[key] for key in dfName])[['End of Week']].drop_duplicates()
        if uniqueDates.shape[0] > 0:
            dfCompleteDates = {}
            dfGroup.append(dfName)
            for key in dfName:
                EndOfWeekcompletDate[key] = pd.merge(uniqueDates, promotionsEndOfWeekCleaned[key], how='left').replace(np.nan, 0).sort_values(by='End of Week').reset_index(drop = True)
    return EndOfWeekcompletDate, dfGroup, dic


In [52]:
if len(categories) != 0:
    dfCategory,catGroup,catDuplication=completeDates(brandMarketCategory,promotionsEndOfWeekCleaned)
if len(sectors) != 0:
    dfSector,secGroup,secDuplication=completeDates(brandMarketSector,promotionsEndOfWeekCleaned)
if len(segments) != 0:
    dfSegment,segGroup,segDuplication=completeDates(brandMarketSegment,promotionsEndOfWeekCleaned)
if len(subsegments) != 0:
    dfSubSegment,subsegGroup,subsegDuplication=completeDates(brandMarketSubSegment,promotionsEndOfWeekCleaned)
if len(subcategories) != 0:
    dfSubCategory,subcatGroup,subcatDuplication=completeDates(brandMarketSubCategory,promotionsEndOfWeekCleaned)

In [53]:
if len(sectors):
    sect_vsod_count =0
    for key,df in sect_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                sect_vsod_count +=1
    sect_vsod_count = sect_vsod_count *len(categories)
 
if len(segments):
    seg_vsod_count =0
    for key,df in seg_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                seg_vsod_count +=1
    seg_vsod_count = seg_vsod_count           
 
if len(subsegments) >0:
    subseg_vsod_count =0
    for key,df in subseg_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                subseg_vsod_count +=1
    subseg_vsod_count = subseg_vsod_count
 
if len(subcategories) >0:
    subcat_vsod_count =0
    for key,df in subcat_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                subcat_vsod_count +=1
    subcat_vsod_count = subcat_vsod_count

In [54]:
PromoRet ={}
if len(categories)!=0:
    first_key, first_value = next(iter(catDuplication.items()))
    PromoRet.update({first_key: first_value})
if len(sectors)!=0:
    sec_key, sec_value = next(iter(secDuplication.items()))
    PromoRet.update({sec_key:sec_value})
if len(segments)!=0:
    third_key, third_value = next(iter(segDuplication.items()))
    PromoRet.update({third_key: third_value})
if len(subsegments)!=0:
    fourth_key, fourth_value = next(iter(subsegDuplication.items()))
    PromoRet.update({fourth_key:fourth_value})
if len(subcategories)!=0:
    fifth_key, fifth_value = next(iter(subcatDuplication.items()))
    PromoRet.update({fifth_key:fifth_value })

In [55]:
PromoSalesTypes_data = {}
for key in brands_promo_type.keys():
    df=brands_promo_type[key].copy()
    df["Promo Sales"] = pd.to_numeric(df["Promo Sales"], errors="coerce").fillna(0)
    df["Value Share"] = pd.to_numeric(df["Value Share"], errors="coerce").fillna(0)
    df = df[df['Promo Type'].notna()]
    brand_totals = df.groupby("Top Brands")['Promo Sales'].sum()
    df["Brand Total Sales"] = df["Top Brands"].map(brand_totals)
    df["% Promo Sales"] = df["Promo Sales"] / df["Brand Total Sales"]

    df = df[~df['Top Brands'].str.contains('Others|Grand Total', case=False)]
    df = df[df['Value Share'] > 0.01]
    df = df[df['Promo Sales'] > 0]
    # Select client brands and additional brands needed to make 10 brands
    df_client = selectClientBrands(brands_promo_type[key],'Top Brands', 'Value Share')
    comp_brand = df[~df['Top Brands'].isin(cb for cb in client_brands)].drop_duplicates("Top Brands")
    if not df_client.empty:
        comp_brand = comp_brand.nlargest(10-df_client["Top Brands"].nunique(), "Value Share")["Top Brands"].to_list()
        # Concatenate client brands and additional brands
        df = df[df["Top Brands"].isin(comp_brand + client_brands)]
        df = df.reset_index(drop=True)
        df = df.sort_values("Value Share", ascending=False).reset_index(drop=True)
        df = df[~df['Promo Type'].fillna('').str.contains('NONE/PL|Undefined|Nan', na=False)]
        # print(comp_brand)
        if df.shape[0]:
            PromoSalesTypes_data[key] =df

In [56]:
final_lis = []
cat_lis = []
if categories:
    for i in range(len(catGroup)):
        cat_lis += genrateIndexList(catGroup[i], chartIndex=14, chartCount=4)[0]
    final_lis.append(cat_lis)
else:
    final_lis.append([])

sec_lis = []
if sectors:
    for i in range(len(secGroup)):
        sec_lis += genrateIndexList(secGroup[i], chartIndex=14, chartCount=4)[0]
    final_lis.append(sec_lis)
else:
    final_lis.append([])

seg_lis=[]
if segments:
    for i in range(len(segGroup)):
        seg_lis += genrateIndexList(segGroup[i], chartIndex=14, chartCount=4)[0]
    final_lis.append(seg_lis)

else:
    final_lis.append([])

subseg_lis =[]
if subsegments:
    for i in range(len(subsegGroup)):
        subseg_lis +=  genrateIndexList(subsegGroup[i], chartIndex=14, chartCount=4)[0]
    final_lis.append(subseg_lis)
else:
    final_lis.append([])

subcat_lis =[]
if subcategories:
    for i in range(len(subcatGroup)):
        subcat_lis +=  genrateIndexList(subcatGroup[i], chartIndex=14, chartCount=4)[0]
    final_lis.append(subcat_lis)
else:
    final_lis.append([])

### New Slide 1

In [57]:
def MonthYear_clean(data,column):
    month_year_data={}
    for key in data.keys():
        df=data[key].copy()
        df = df[~((df[column].str.contains('Total', case=False)) & (df[column] != categories[0]))].reset_index(drop=True)

        # df = df[~df[column].str.contains('Total', case=False)].reset_index(drop=True)
        df['year'] = pd.to_datetime(df['MonthYear'], format='%b-%y').dt.year
        df['year'] = df['year'].astype(int)
        yearly_avg_sales = df.groupby(['year', column])['Value Sales'].transform('mean').reset_index(drop=True) 
        yearly_avg_sales = yearly_avg_sales.replace(0, float('nan'))
        # Calculate 'Sales index' and handle NaN values gracefully
        df['Sales index'] = (df['Value Sales'] / yearly_avg_sales * 100).fillna(0).astype(int)
        if df.shape[0]>0:
            month_year_data[key] = df
    return month_year_data

In [58]:
category_month_year=MonthYear_clean(Category_MonthYear,column='Category')
sector_month_year = MonthYear_clean(Sector_MonthYear,column='Sector')
segment_month_year = MonthYear_clean(Segment_MonthYear,column='Segment')
subcat_month_year = MonthYear_clean(SubCategory_MonthYear,column='SubCategory')
subseg_month_year = MonthYear_clean(SubSegment_MonthYear,column='SubSegment')

In [59]:
def split_month_year(data, column):
    final_month_year ={}
    for key,df in data.items():
        for sec in df[column].unique():
            newkey = key + ' | ' + sec
            new_df = df[df[column] == sec].reset_index(drop=True)
            if new_df.shape[0] > 0:
                final_month_year[newkey] = new_df
    return final_month_year

In [60]:
category_month_year1=split_month_year(category_month_year,'Category')
sector_month_year1 = split_month_year(sector_month_year,'Sector')
segment_month_year1 = split_month_year(segment_month_year,'Segment')
subseg_month_year1 = split_month_year(subseg_month_year,'SubSegment')
subcat_month_year1 = split_month_year(subcat_month_year,'SubCategory')

In [61]:
month_year1 = {}
month_year1.update(sector_month_year1)
month_year1.update(segment_month_year1)
month_year1.update(subcat_month_year1)
month_year1.update(subseg_month_year1)

In [62]:
brandMarketCategory = [key for key in category_month_year1.keys() if any(cat == key.split(' | ')[2]  for cat in categories )]
if len(sectors) != 0:
    brandMarketSector = [key for key in sector_month_year1.keys() if any(cat == key.split(' | ')[2]  for cat in sectors )]
if len(segments) != 0:
    brandMarketSegment = [key for key in segment_month_year1.keys() if any(cat == key.split(' | ')[2]  for cat in segments )]
if len(subsegments) != 0:
    brandMarketSubSegment = [key for key in subseg_month_year1.keys() if any(cat == key.split(' | ')[2] for cat in subsegments )]
if len(subcategories) != 0:
    brandMarketSubCategory = [key for key in subcat_month_year1.keys() if any(cat == key.split(' | ')[2] for cat in subcategories )]

In [63]:
def completeDates1(dfList, promotionsEndOfWeekCleaned,column="Sector"):
    """
    Complete dates for each dataframe in dfList based on promotionsEndOfWeekCleaned.

    Parameters:
    dfList (list): List of dataframe keys.
    promotionsEndOfWeekCleaned (dict): Dictionary containing cleaned promotions end of week data.

    Returns:
    tuple: Tuple containing EndOfWeekcompletDate (dictionary), dfGroup (list), and dic (dictionary).
    """
    # Create a list of unique brand-category combinations
    brandCatList = sorted(set(key.split(' | ')[0]  for key in dfList))
    # Initialize dictionaries and lists
    EndOfWeekcompletDate = {}
    dfGroup = []
    dic = defaultdict(int)
    
    # Count occurrences of each brand-category combination
    for key in brandCatList:
        for name in dfList:
            if (key.split(' | ')[0] == name.split(' | ')[0]):
                
                dic[key] += 1
                
    # Iterate over unique brand-category combinations
    for name in dic.keys():

        if column == "Sector" :
            
            # Filter dataframe keys associated with the current brand-category combination
            dfName = [key for key in dfList if name == key.split(' | ')[0] and len(name.split(' | ')) == 1]
        else:
            
            dfName = [key for key in dfList if name == key.split(' | ')[0]  ]
        
        # Extract unique dates from all associated dataframes
        uniqueDates = pd.concat([promotionsEndOfWeekCleaned[key] for key in dfName])[['MonthYear']].drop_duplicates()
        # Initialize dictionary for complete dates for each dataframe
        dfCompleteDates = {}
        # Add dataframe keys to the group list
        dfGroup.append(dfName)
        # Populate EndOfWeekcompletDate dictionary with dataframes merged on unique dates
        for key in dfName:
            EndOfWeekcompletDate[key] = pd.merge(uniqueDates, promotionsEndOfWeekCleaned[key], how='left')#.replace(np.nan, 0)
            column = EndOfWeekcompletDate[key].columns[1]
            year = EndOfWeekcompletDate[key].columns[3]
            monthyear = EndOfWeekcompletDate[key].columns[0]
            EndOfWeekcompletDate[key][column] = EndOfWeekcompletDate[key][column].fillna(method='ffill')      
            EndOfWeekcompletDate[key][year] = pd.to_datetime(EndOfWeekcompletDate[key][monthyear], format='%b-%y').dt.year
            EndOfWeekcompletDate[key] = EndOfWeekcompletDate[key].fillna(0)
    return EndOfWeekcompletDate, dfGroup, dic

In [64]:
dfCategory0,categoryGroup0,categoryDuplication0=completeDates1(brandMarketCategory,category_month_year1,column="Category")
if len(sectors) != 0:
    dfSector0,secGroup0,secDuplication0=completeDates1(brandMarketSector,sector_month_year1,column="Sector")
if len(segments) != 0:
    dfSegment0,segGroup0,segDuplication0=completeDates1(brandMarketSegment,segment_month_year1,column="Segment")
if len(subsegments) != 0:
    dfSubSegment0,subsegGroup0,subsegDuplication0=completeDates1(brandMarketSubSegment,subseg_month_year1,column="Subsegment")
if len(subcategories) != 0:
    dfSubCategory0,subcatGroup0,subcatDuplication0=completeDates1(brandMarketSubCategory,subcat_month_year1,column="Subcategory")

In [65]:
def groupingkeys(data):
    grouped = defaultdict(list)

    for sublist in data:
        for entry in sublist:
            prefix = " | ".join(entry.split(" | ")[:2])  # Extract first two parts
            grouped[prefix].append(entry)
    result = list(grouped.values())
    return result

In [66]:
categoryGroup0=groupingkeys(categoryGroup0)
if len(sectors) != 0:
    secGroup0=groupingkeys(secGroup0)
if len(segments) != 0:
    segGroup0=groupingkeys(segGroup0)
if len(subsegments) != 0:
    subsegGroup0=groupingkeys(subsegGroup0)
if len(subcategories) != 0:
    subcatGroup0=groupingkeys(subcatGroup0)

In [67]:
final_lis0 = []
category_lis = []
if categories:
    for i in range(len(categoryGroup0)):
        category_lis += genrateIndexList(categoryGroup0[i], chartIndex=19, chartCount=6)[0]
final_lis0.append(category_lis)  # Append empty list if sectors is False

sec_lis = []
if sectors:
    for i in range(len(secGroup0)):
        sec_lis += genrateIndexList(secGroup0[i], chartIndex=19, chartCount=6)[0]
final_lis0.append(sec_lis)  # Append empty list if sectors is False
print(final_lis0)

seg_lis = []
if segments:
    for i in range(len(segGroup0)):
        seg_lis += genrateIndexList(segGroup0[i], chartIndex=19, chartCount=6)[0]
final_lis0.append(seg_lis)  # Append empty list if segments is False
print(final_lis0)

subseg_lis = []
if subsegments:
    for i in range(len(subsegGroup0)):
        subseg_lis += genrateIndexList(subsegGroup0[i], chartIndex=19, chartCount=6)[0]
final_lis0.append(subseg_lis)  # Append empty list if subsegments is False
print(final_lis0)

subcat_lis = []
if subcategories:
    for i in range(len(subcatGroup0)):
        subcat_lis += genrateIndexList(subcatGroup0[i], chartIndex=19, chartCount=6)[0]
final_lis0.append(subcat_lis)  # Append empty list if subcategories is False


[[20, 20], [21, 21]]
[[20, 20], [21, 21], [20, 20, 21, 21]]
[[20, 20], [21, 21], [20, 20, 21, 21], []]


### New slide 2

In [68]:
modified_endofweek_P12M = {}
past_12_months = pd.date_range(end=end_date , periods=12, freq='M').strftime('%b-%y').tolist()
for key in modified_promotionEndOfWeek.keys():
    df=modified_promotionEndOfWeek[key].copy()
    df['End of Week'] = pd.to_datetime(df['End of Week'])
    filtered_df = df[df['End of Week'].dt.strftime('%b-%y').isin(past_12_months)]
    if filtered_df.shape[0] >0:
        modified_endofweek_P12M[key] = filtered_df 

In [69]:
brandMarketCategory= [key for key in modified_endofweek_P12M.keys() if any(cat in key.split(' | ')[0] for cat in categories )]
if len(sectors) != 0:
    brandMarketSector = [key for key in modified_endofweek_P12M.keys() if any(cat == key.split(' | ')[0] for cat in sectors )]
if len(segments) != 0:
    brandMarketSegment = [key for key in modified_endofweek_P12M.keys() if any(cat == key.split(' | ')[0] for cat in segments )]
if len(subsegments) != 0:
    brandMarketSubSegment = [key for key in modified_endofweek_P12M.keys() if any(cat == key.split(' | ')[0] for cat in subsegments )]
if len(subcategories) != 0:
    brandMarketSubCategory = [key for key in modified_endofweek_P12M.keys() if any(cat == key.split(' | ')[0] for cat in subcategories )]

In [70]:
if len(categories) != 0:
    dfCategory1,catGroup1,catDuplication1=completeDates(brandMarketCategory,modified_endofweek_P12M)
if len(sectors) != 0:
    dfSector1,secGroup1,secDuplication1=completeDates(brandMarketSector,modified_endofweek_P12M)
if len(segments) != 0:
    dfSegment1,segGroup1,segDuplication1=completeDates(brandMarketSegment,modified_endofweek_P12M)
if len(subsegments) != 0:
    dfSubSegment1,subsegGroup1,subsegDuplication1=completeDates(brandMarketSubSegment,modified_endofweek_P12M)
if len(subcategories) != 0:
    dfSubCategory1,subcatGroup1,subcatDuplication1=completeDates(brandMarketSubCategory,modified_endofweek_P12M)

In [71]:
def promofrequencyclean(data):        
        modified_dfCategory1 = {}
        for k in data.keys():
                chart_df=data[k].copy()
                chart_df['Weekly VSOD'] = np.where((chart_df['VSOD']>.2)&(chart_df['Value Uplift (v. base) Normalized'] != ''),1,None)
                chart_df['try'] = 0
                chart_df['New Uplift'] = 0
                chart_df['try'] = np.where((chart_df['Value Uplift (v. base) Normalized']>=2),1.8,chart_df['Value Uplift (v. base) Normalized'])
                chart_df['New Uplift'] = np.where((chart_df['Weekly VSOD']==1)&(chart_df['Value Uplift (v. base) Normalized']>0.05),chart_df['try'],None)
                if not chart_df['Weekly VSOD'].isnull().all():
                        modified_dfCategory1[k]= chart_df 
        return modified_dfCategory1

In [72]:
if len(categories)!=0: 
    modified_dfCategory1=promofrequencyclean(dfCategory1)
if len(sectors)!=0: 
    modified_dfSector1=promofrequencyclean(dfSector1)
if len(segments)!=0: 
    modified_dfSegment1=promofrequencyclean(dfSegment1)
if len(subsegments)!=0: 
    modified_dfSubSegment1=promofrequencyclean(dfSubSegment1)
if len(subcategories)!=0: 
    modified_dfSubCategory1=promofrequencyclean(dfSubCategory1)

In [73]:
brandMarketCategory= [key for key in modified_dfCategory1.keys() if any(cat in key.split(' | ')[0] for cat in categories )]
if len(sectors) != 0:
    brandMarketSector = [key for key in modified_dfSector1.keys() if any(cat == key.split(' | ')[0] for cat in sectors )]
if len(segments) != 0:
    brandMarketSegment = [key for key in modified_dfSegment1.keys() if any(cat == key.split(' | ')[0] for cat in segments )]
if len(subsegments) != 0:
    brandMarketSubSegment = [key for key in modified_dfSubSegment1.keys() if any(cat == key.split(' | ')[0] for cat in subsegments )]
if len(subcategories) != 0:
    brandMarketSubCategory = [key for key in modified_dfSubCategory1.keys() if any(cat == key.split(' | ')[0] for cat in subcategories )]

In [74]:
if len(categories) != 0:
    dfCategory1,catGroup1,catDuplication1=completeDates(brandMarketCategory,modified_endofweek_P12M)
if len(sectors) != 0:
    dfSector1,secGroup1,secDuplication1=completeDates(brandMarketSector,modified_endofweek_P12M)
if len(segments) != 0:
    dfSegment1,segGroup1,segDuplication1=completeDates(brandMarketSegment,modified_endofweek_P12M)
if len(subsegments) != 0:
    dfSubSegment1,subsegGroup1,subsegDuplication1=completeDates(brandMarketSubSegment,modified_endofweek_P12M)
if len(subcategories) != 0:
    dfSubCategory1,subcatGroup1,subcatDuplication1=completeDates(brandMarketSubCategory,modified_endofweek_P12M)

In [75]:
def group_by_region(data):
    # Define categories for grouping
    market_groups = {
        "RETAILER_REGIONS": regions_RET,
        "RETAILER_CHANNELS": channels_RET,
        "RETAILER_MARKET": market_RET,
        "CHANNEL_REGIONS": regions_CHAN,
        "CHANNEL_CHANNELS": channels_CHAN,
        "CHANNEL_MARKET": market_CHAN,
        f"{customareas}_REGIONS": regions_CUST,
        f"{customareas}_CHANNELS": channels_CUST,
        f"{customareas}_MARKET": market_CUST,
    }
    result = []
    for sublist in data:
        for category, keywords in market_groups.items():
            # Filter items matching the current category
            base_category = category.split("_")[0]

            group = [
                f"{item} | {base_category}" for item in sublist if item.split(" | ")[-1] in keywords
            ]
            if group:  # Append only non-empty groups
                result.append(group)
    return result


In [76]:
if len(categories)>0:
    catGroup1 = group_by_region(catGroup1)
if len(sectors)>0:
    secGroup1 = group_by_region(secGroup1)
if len(segments)>0:
    segGroup1 = group_by_region(segGroup1)
if len(subsegments)>0:
    subsegGroup1 = group_by_region(subsegGroup1)
if len(subcategories)>0:
    subcatGroup1 = group_by_region(subcatGroup1)

In [77]:
final_lis1 = []
cat_lis = []
if categories:
    for i in range(len(catGroup1)):
        cat_lis += genrateIndexList(catGroup1[i], chartIndex=25, chartCount=4)[0]
    final_lis1.append(cat_lis)
else:
    final_lis1.append([])

sec_lis = []
if sectors:
    for i in range(len(secGroup1)):
        sec_lis += genrateIndexList(secGroup1[i], chartIndex=25, chartCount=4)[0]
    final_lis1.append(sec_lis)
else:
    final_lis1.append([])

seg_lis=[]
if segments:
    for i in range(len(segGroup1)):
        seg_lis += genrateIndexList(segGroup1[i], chartIndex=25, chartCount=4)[0]
    final_lis1.append(seg_lis)

else:
    final_lis1.append([])

subseg_lis =[]
if subsegments:
    for i in range(len(subsegGroup1)):
        subseg_lis +=  genrateIndexList(subsegGroup1[i], chartIndex=25, chartCount=4)[0]
    final_lis1.append(subseg_lis)
else:
    final_lis1.append([])

subcat_lis =[]
if subcategories:
    for i in range(len(subcatGroup1)):
        subcat_lis +=  genrateIndexList(subcatGroup1[i], chartIndex=25, chartCount=4)[0]
    final_lis1.append(subcat_lis)
else:
    final_lis1.append([])

In [78]:
retailer=regions_RET+channels_RET+market_RET
channels=regions_CHAN+channels_CHAN+channels_CHAN
customarea=regions_CUST+channels_CUST+market_CUST
def addarea(modified_dfCategory1,retailer,market="RETAILER"):
    keys_to_modify = [k for k in modified_dfCategory1.keys() if k.split(" | ")[-1] in retailer]
    for k in keys_to_modify:
        new_key = k + " | "+ market
        modified_dfCategory1[new_key] = modified_dfCategory1[k]  
        del modified_dfCategory1[k]       
    return modified_dfCategory1      
if len(categories)>0:            
    modified_dfCategory1=addarea(modified_dfCategory1,retailer,market="RETAILER")
    modified_dfCategory1=addarea(modified_dfCategory1,channels,market="CHANNELS")
    modified_dfCategory1=addarea(modified_dfCategory1,customarea,market=f"{customareas}")

if len(sectors)>0:            
    modified_dfSector1=addarea(modified_dfSector1,retailer,market="RETAILER")
    modified_dfSector1=addarea(modified_dfSector1,channels,market="CHANNELS")
    modified_dfSector1=addarea(modified_dfSector1,customarea,market=f"{customareas}")
if len(segments)>0:            
    modified_dfSegment1=addarea(modified_dfSegment1,retailer,market="RETAILER")
    modified_dfSegment1=addarea(modified_dfSegment1,channels,market="CHANNELS")
    modified_dfSegment1=addarea(modified_dfSegment1,customarea,market=f"{customareas}")
if len(subsegments)>0:            
    modified_dfSubSegment1=addarea(modified_dfSubSegment1,retailer,market="RETAILER")
    modified_dfSubSegment1=addarea(modified_dfSubSegment1,channels,market="CHANNELS")
    modified_dfSubSegment1=addarea(modified_dfSubSegment1,customarea,market=f"{customareas}")
if len(subcategories)>0:            
    modified_dfSubCategory1=addarea(modified_dfSubCategory1,retailer,market="RETAILER")
    modified_dfSubCategory1=addarea(modified_dfSubCategory1,channels,market="CHANNELS")
    modified_dfSubCategory1=addarea(modified_dfSubCategory1,customarea,market=f"{customareas}")


In [79]:
for k in list(modified_promotionBrandsP12M.keys()):  # Convert to list to avoid runtime errors
    df = modified_promotionBrandsP12M[k].copy()
    # Filter rows based on 'Top Brands'
    df = df[~df['Top Brands'].str.contains('Others', case=False, na=False)]
    df = df[~df['Top Brands'].str.contains('Grand Total', case=False, na=False)]
    df = df[df['Value Share'] > 0.01]
    if not df.empty:
        modified_promotionBrandsP12M[k] = df
    else:
        del modified_promotionBrandsP12M[k]



## Slide duplication: index, duplication and section names

In [80]:
index = [*[0]*5,
         #*[1]*5,
         *[1]*5,
         *[2]*5,
         *[3]*5,
         *[4]*5,
         *[5]*5,
         *[6]*5,
         *[7]*5,
         *[8]*4,
         *[9]*5,
         *[10]*5,
         *[11]*5,
         *[12]*5,
         *[13]*5,
         *[14]*5,
         *final_lis,
         *[19]*5,
         *final_lis0,
         *final_lis1,
         *[0]*5,
         *[1]*5,
         *[2]*5,
         *[9]*5,
         *[10]*5,
        #  *[11]*5,
         *[12]*5,
         *[13]*5
         #*[14]*5
        ]
duplication = combine_duplications(Scope,count_df,[#modified_promotionBrandsP12M, #0
                                                   promotionsBrandSortedTotalFinal, #1
                                                   newpromotionsNotBrandsWithMarket, #2
                                                   concated, #3
                                                   modified_promotionProductsP12M_volumeuplift, #4
                                                   new_modified_promotionProductsP12M, #5
                                                   new_modified_promotionProductsP12M, #6
                                                   top20clientonly, #7
                                                   bottom20clientonly,#8
                                                   modified_promotionBrandsP12M, #10
                                                   newModifiedBrands, #11
                                                   PromoSalesTypes_data if promo_type else None,#12
                                                   modified_promotionBrandsP12M if feature_share else None, #13
                                                   modified_promotionBrandsP12M if display_share else None, #14
                                                   modified_promotionEndOfWeek,#15
                                                   PromoRet, #16-19
                                                   modified_valueUplift, #20
                                                   #month_year1,#21
                                                   #modified_endofweek_P12M, #22
                                                   #modified_promotionBrandsP12M, #0 with no client
                                                   promotionsBrandNOTSortedTotalFinal, #1 with no client
                                                   newpromotionsNotBrandsWithMarket, #2 with no client
                                                   concated, #3 with no client
                                                   modified_promotionBrandsP12M, # 10 with no client
                                                   newModifiedBrands, #11 with no client
                                                   #PromoSalesTypes_data if promo_type else None,#12  with no client
                                                   modified_promotionBrandsP12M if feature_share else None, #13  with no client
                                                   modified_promotionBrandsP12M if display_share else None #14 with no client
                                                  ])
section_names = [#"Promo Value Sale",#0
                 "Promo Evolution", #1
                 "VSOD Summary by Sector" , #2
                 "Value uplift by retailer by brand", #3 
                 "Volume Uplift vs discount depth",#4
                 "Value Uplift vs Promo Efficiency Quadrant", #5
                 "Top 20 promotions", #6
                 "Top 20 promotions CLIENT ONLY", #7
                 "Bottom 20 promotions CLIENT ONLY", #8
                 "Promo share vs Value Share", #10
                 "Promo Sales by total size",#11
                 "Promo Sales by promo type", #12
                 "Feature Share vs Fair Share", #13
                 "Display Share vs Fair Share", #14
                 "Promo Frequency learnings", #15
                 "Promo sales per retailer", #16-19
                 "Value Uplift vs discount depth" ,#20
                 #"Seasonality Index",#21
                 #"Promotional Frequency Analysis", #22
                 #"Promo Value Sale no client prio",
                 "Promo Evolution no client prio",
                 "VSOD Summary by Sector no client prio",
                 "Value uplift by retailer by brand no client prio",
                "Promo share vs Value Share no client prio", #10
                 "Promo Sales by total size no client prio",#11
                 #"Promo Sales by promo type no client prio", #12
                 "Feature Share vs Fair Share no client prio", #13
                 "Display Share vs Fair Share no client prio" #14
                ]



#duplication.insert(89, 0)

if len(sectors) > 0:
       #duplication.insert(45,(len(client_manuf)+len(client_brands))*len(categories)* len(marketList))
       duplication.insert(40, sect_vsod_count)
if len(segments) > 0:
        #duplication.insert(46,(len(client_manuf)+len(client_brands))*len(sectors)* len(marketList)) 
         duplication.insert(41, seg_vsod_count)
 
else:
    duplication.insert(41,0)  
  
if len(subsegments) > 0:
        #duplication.insert(47,(len(client_manuf)+len(client_brands))*len(segments)* len(marketList))
        duplication.insert(42, subseg_vsod_count)

else:
    duplication.insert(42,0)

if len(subcategories) > 0:
        #duplication.insert(48,(len(client_manuf)+len(client_brands))*len(segments)* len(marketList))
        duplication.insert(43, subcat_vsod_count)

else:
    duplication.insert(43,0)


duplication.insert(84,1)
duplication.insert(85, 1)
duplication.insert(86, 1)
duplication.insert(87, 1)
duplication.insert(88, 0)
duplication.insert(89, 1)
duplication.insert(90, 1)
duplication.insert(91, 1)
duplication.insert(92, 1)
duplication.insert(93, 0)

section_names = [f"{name} {suffix}" for name in section_names for suffix in suffixes]

section_names.insert(40,'Volume Sold on Deal Sector')
section_names.insert(41,'Volume Sold on Deal Segment')
section_names.insert(42,'Volume Sold on Deal SubSegment')
section_names.insert(43,'Volume Sold on Deal SubCategory')

section_names.insert(84,'Seasonality Index Category')
section_names.insert(85,'Seasonality Index Sector')
section_names.insert(86,'Seasonality Index Segment')
section_names.insert(87,'Seasonality Index Subsegment')
section_names.insert(88,'Seasonality Index Subcategory')

section_names.insert(89,'Promotional Frequency Analysis Category')
section_names.insert(90,'Promotional Frequency Analysis Sector')
section_names.insert(91,'Promotional Frequency Analysis Segment')
section_names.insert(92,'Promotional Frequency Analysis Subsegment')
section_names.insert(93,'Promotional Frequency Analysis Subcategory')
#section_names.insert(94,'Promotional Frequency Analysis Subcategory')

duplication[77]=1
#index = [i for i in index if i != []]
# duplication = [i for i in duplication if i != []]
#

path = os.getcwd() + '//Promotion base.pptx'
new_pre = os.getcwd() + '//slide duplicated.pptx'


### Deck 1

In [81]:
print(index)
print(duplication)
print(section_names)
print(len(index))
print(len(duplication))
print(len(section_names))
print(final_lis)
print(final_lis1)
print(sum(duplication))

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, [17, 18], [16, 17, 16, 18], [16, 17, 16, 18, 18], [], [], 19, 19, 19, 19, 19, [20, 20], [21, 21], [20, 20, 21, 21], [], [], [28, 28], [27, 28, 27, 29], [27, 28, 27, 28, 29], [], [], 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13]
[2, 4, 6, 0, 0, 0, 2, 2, 0, 0, 1, 2, 3, 0, 0, 2, 4, 6, 0, 0, 2, 4, 6, 0, 0, 2, 4, 6, 0, 0, 2, 4, 5, 0, 0, 2, 4, 5, 0, 0, 7, 14, 0, 0, 2, 4, 6, 0, 0, 1, 2, 3, 0, 0, 2, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 11, 15, 0, 0, 1, 1, 1, 1, 0, 5, 7, 10, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 4, 6, 0, 0, 0, 2, 2, 0, 0, 1, 2, 3, 0, 0, 2, 4, 6, 0, 0, 1, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['Promo Evolution Category', 'Promo Evolution Sector', 'Pr

In [82]:
slideDuplication(index,duplication,section_names,path,new_pre)

## Replace duplicated slides with data

In [ ]:
prs = Presentation(new_pre)
posItr = 0
ind =0

In [ ]:
def promoValueSales(prs, promotionsBrandDF, numOfDuplicates, position=0):
    """
    Generate PowerPoint slides for promo value sales.

    Args:
    - prs (pptx.presentation): PowerPoint presentation object.
    - promotionsBrandDF (dict): Dictionary of DataFrames containing promotion data for different markets.
    - numOfDuplicates (int): Number of slides to duplicate for different markets.
    - position (int): Position to start adding slides in the presentation.

    Returns:
    - Replace the slides with new data
    """
    # Loop through each slide number
    slidenum = 0
    for key,df in promotionsBrandDF.items():
        # Retrieve DataFrame for the current market
        df = promotionsBrandDF[key].reset_index(drop=True)
        
        # Remove rows with 'Others' in 'Top Brands' column and filter by 'Value Share'
        df = df[~df['Top Brands'].str.contains('Others', case=False)]
        df = df[~df['Top Brands'].str.contains('Grand Total', case=False)]
        df = df[df['Value Share'] > 0.01]
     
  
        # Select client brands
        df_client = selectClientBrands(df,'Top Brands', 'Promo Value')
        number_of_brands_needed = max(6 - len(df_client),0)
     
        # Filter top brands and concatenate with client brands
        df = df[~df['Top Brands'].isin(client_brands)]
        df = df.sort_values(by='Promo Value', ascending=False).head(number_of_brands_needed)
        df = pd.concat([df, df_client], ignore_index=True)
        df = df.sort_values(by='Promo Value', ascending=False)
    
        
        # Update title
        shapes = prs.slides[slidenum + position].shapes
        titlNumber = get_shape_number(shapes, "Promo Value Sales | Category | National | P12M")
        headerNumber = get_shape_number(shapes, "Promo Value Sales (Replace With SO WHAT)")
        shapes[titlNumber - 1].text = data_source
        shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
        shapes[titlNumber].text = shapes[titlNumber].text.replace('Category', key.split(' | ')[0]).replace(
            'National', key.split(' | ')[1])
        shapes[titlNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titlNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
        
        # Create table and chart
        tables, charts = createTableAndChart(shapes)
        table = tables[0].table
        
        # Remove unnecessary rows
        num_rows_to_remove = len(table.rows) - df['Top Brands'].nunique() - 1
        table_height = get_table_height(table)
        for _ in range(num_rows_to_remove):
            if len(table.rows) > 1:
                row = table.rows[1]
                remove_row(table, row)
        
        # Adjust row heights
        total_row_height = table_height - table.rows[0].height
        num_rows = len(table.rows) - 1
        if num_rows > 0:
            cell_height = total_row_height / num_rows
            for row in range(1, table.rows.__len__()):
                table.rows[row].height = int(cell_height)
        
        # Populate table cells
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if i == 0:  # Header row
                    continue
                if j == 0:  # Brand column
                    cell.text = df['Top Brands'].iloc[i - 1]
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                elif j == 1:  # Promo Value sales column
                    value = df['Promo Value'].iloc[i - 1]
                    if len(str(value)) > 3:
                        formatted_value = '{:,}'.format(int(value))
                        cell.text = str(formatted_value)
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                    else:
                        cell.text = str(df['Promo Value'].iloc[i - 1])
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                elif j == 2:  # Volume Sold on Deal (VSOD) column
                    cell.text = str(int(round(df['VSOD'].replace(np.nan, 0).iloc[i - 1] * 100, 0))) + '%'
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                else:  # VSOD IYA column
                    cell.text = str(int(round(df['VSOD IYA'].replace(np.nan, 0).iloc[i - 1] * 100, 0)))
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                # Set font size and alignment
                cell.text_frame.paragraphs[0].font.size = Pt(8)
                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
        slidenum +=1


In [ ]:
# slide 2
for key,value in Scope.items():
    dict = {key: count_df(promotionsBrandSortedTotalFinal,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in promotionsBrandSortedTotalFinal.items() if key in dict[key1]}
        if filtered_dict:
            promoEvolutionNew(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1


In [ ]:
print(ind,posItr)

5 12


In [ ]:
# slide 3
for key,value in Scope.items():
    dict = {key: count_df(newpromotionsBrandsWithMarket,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in newpromotionsBrandsWithMarket.items() if key in dict[key1]}
        if filtered_dict:
            VSOD1(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

10 16


In [ ]:
def valueUpliftRetailer(prs, concated, numOfDuplicates, position=0):
    """
    This function updates a PowerPoint presentation with value uplift by brand for different categories/sectors.
    It modifies the slide shapes and populates tables and charts with the value uplift data.
 
    Parameters:
    prs (Presentation): The PowerPoint presentation object to modify.
    concated (dict): A dictionary where keys are category/sector names and values are DataFrames containing value uplift data.
    numOfDuplicates (int): The number of slides to duplicate and update.
    position (int, optional): The starting slide position in the presentation. Defaults to 0.
 
    Returns:
    Replace the slides with new data
    """
    for key, slide_num in zip(concated, range(numOfDuplicates)):
        # Get the DataFrame for the current category/sector
        df = concated[key]
        df = df[df['Promo Value'] > 0].reset_index(drop=True)
        df = df[~df['Top Brands'].str.contains('Grand Total', case=False)]
 
        # Get the current slide and its shapes
        slide = prs.slides[slide_num + position]
        shapes = slide.shapes
       
        # Update the title shape with the category/sector name
        titleNumber = get_shape_number(shapes, "Value Uplift by brand | Category/Sector | P12M")
        headereNumber = get_shape_number(shapes, "Value uplift by retailer by brand (Replace With SO WHAT)")
        shapes[titleNumber-1].text = data_source
        shapes[titleNumber].text = shapes[titleNumber].text.replace('Category/Sector', key)
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
        shapes[headereNumber].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[headereNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
 
        # Create and retrieve table and chart shapes
        tables, charts = createTableAndChart(slide.shapes)
        chart = charts[0].chart
        table = tables[0].table
       
        # Get the list of markets (sources)
        market_list = list(df['SOURCE'].unique())
       
        # Adjust the number of columns in the table
        num_cols_to_remove = len(table.columns) - len(market_list)
        table_width = get_table_width(table)
        for _ in range(num_cols_to_remove):
            if len(table.columns):  # Skip removing the first row if there is more than one row
                col = table.columns[0]
                remove_col(table, col)
        if num_cols_to_remove:
            total_col_width = table_width
            num_cols = len(table.columns)
            if num_cols > 0:
                cell_width = total_col_width / num_cols
                for col in range(0, table.columns.__len__()):
                    table.columns[col].width = int(cell_width)
 
        # Populate the table with market names
        for row_number, row in enumerate(table.rows, start=0):
            for column_num, cell in enumerate(row.cells):
                value = market_list[column_num]
                cell.text = str(value)
                set_cell_font(cell, 'Nexa Bold', 8)
                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
                cell.text_frame.paragraphs[0].font.bold = False
       
        # Prepare chart data 
        chart_data = BubbleChartData()
        chart_data.categories = [i for i in range((df['SOURCE'].nunique() * 2) + 1)]
        series = chart_data.add_series('Average Value Uplift')
        catPos = [i for i in range(1, len(chart_data.categories), 2)]
 
        # Populate the chart with value uplift data
        filtered_brands_list = []

        for i, source in enumerate(df['SOURCE'].unique()):
            dfSource = df[df['SOURCE'] == source]
            dfSource = dfSource[~dfSource['Top Brands'].str.contains('Others', case=False)]
            dfSource = dfSource[dfSource['Value Share'] > 0.01]
            df_client = selectClientBrands(dfSource,'Top Brands', 'Value Share')
            number_of_brands_needed = max(10 - len(df_client),0)
            dfSource = dfSource[~dfSource['Top Brands'].isin(client_brands)]
            dfSource = dfSource.sort_values(by='Value Share', ascending=False).head(number_of_brands_needed)
            dfSource = pd.concat([dfSource, df_client], ignore_index=True)
            series.has_data_labels = True
            start = len(series)
            brands = dfSource['Top Brands'].unique()
             # ✅ Collect filtered brands
            if isinstance(dfSource, pd.DataFrame) and 'Top Brands' in dfSource.columns:
                filtered_brands_list.append(dfSource[['Top Brands']].copy())
            # print(key,filtered_brands_list)
# Concatenate all DataFrames and get unique brand names
            all_top_brands = pd.concat(filtered_brands_list, ignore_index=True)
            # unique_brands = all_top_brands['Top Brands'].tolist()

            # print(key, filtered_brands_list)

            # brands = dfSource['Top Brands'].unique()
            # brandslis = dfSource['Top Brands'].tolist()
            



            for brand in brands:
                if normalized:
                    series.add_data_point(catPos[i], dfSource[dfSource['Top Brands'] == brand]['Value Uplift (v. base) Normalized'].unique()[0], dfSource[dfSource['Top Brands'] == brand]['Promo Value'].unique()[0])
                else: 
                    series.add_data_point(catPos[i], dfSource[dfSource['Top Brands'] == brand]['Value Uplift (v. base)'].unique()[0], dfSource[dfSource['Top Brands'] == brand]['Promo Value'].unique()[0])
            chart.replace_data(chart_data)
            value_axis = chart.category_axis
            value_axis.minimum_scale = 0
            value_axis.maximum_scale = (df['SOURCE'].nunique() * 2)
 
            # chart.replace_data(chart_data)
            
            # all_top_brands = pd.concat(filtered_brands_list, ignore_index=True)
            
            # print(f"{key}: All Top Brands after filtering and concat:\n", all_top_brands['Top Brands'])
            xlsx_file=BytesIO()
            with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                worksheet.write(0, 4, "labels")
                worksheet.write_column(1, 4,all_top_brands['Top Brands'], None)
            chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())
 
            stop = len(chart.series[0].points)
            clibrands=all_top_brands['Top Brands']
            for j in range(start, stop):
                point = chart.series[0].points[j]
                # data_label = point.data_label
                # data_label.has_text_frame = True
                # data_label.text_frame.text = brands[j % len(brands)]
                # data_label.text_frame.paragraphs[0].runs[0].font.size = Pt(8)
                # data_label.position = XL_LABEL_POSITION.CENTER
                point.format.fill.solid()
                if clibrands[j % len(clibrands)] in client_brands:
                    point.format.fill.fore_color.rgb = RGBColor(126, 202, 196)
                else:
                    point.format.fill.fore_color.rgb = RGBColor(230, 229, 229)
 
        # Set the axis scale for the chart
        value_axis = chart.category_axis
        value_axis.minimum_scale = 0
        value_axis.maximum_scale = (df['SOURCE'].nunique() * 2)
 
        # chart.replace_data(chart_data)

In [ ]:
# slide 4
for key,value in Scope.items():
    dict = {key: count_df(concated,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in concated.items() if key in dict[key1]}
        if filtered_dict:
            valueUpliftRetailer(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

15 22


In [ ]:
# slide 5
for key,value in Scope.items():
    dict = {key: count_df(modified_promotionProductsP12M_volumeuplift,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in modified_promotionProductsP12M_volumeuplift.items() if key in dict[key1]}
        if filtered_dict:
            VolumeUplift(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:

print(ind,posItr)

20 34


In [ ]:
# slide 6
for key,value in Scope.items():
    dict = {key: count_df(new_modified_promotionProductsP12M,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in new_modified_promotionProductsP12M.items() if key in dict[key1]}
        if filtered_dict:
            ValueUpliftvsPromoEfficiencyQuadrant(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

25 46


In [ ]:
# slide 7
for key,value in Scope.items():
    dict = {key: count_df(new_modified_promotionProductsP12M,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in new_modified_promotionProductsP12M.items() if key in dict[key1]}
        if filtered_dict:
            top20(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

30 58


In [ ]:
# slide 8
for key,value in Scope.items():
    dict = {key: count_df(top20clientonly,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in top20clientonly.items() if key in dict[key1]}
        if filtered_dict:
            top20Client(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

35 69


In [ ]:
# slide 9
for key,value in Scope.items():
    dict = {key: count_df(bottom20clientonly,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in bottom20clientonly.items() if key in dict[key1]}
        if filtered_dict:
            bot20Client(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

40 80


In [ ]:
if len(sectors)>0:
    newVolumeSold(prs, sect_vsod_merged, position=posItr, parent=direct_parent['Sector'], child = 'Sector')
    print(posItr)
    posItr += sect_vsod_count
    ind +=1
else:
    ind+=1

80


In [ ]:
print(ind,posItr)

41 87


In [ ]:
if len(segments)>0:
    newVolumeSold(prs, seg_vsod_merged, position=posItr, parent=direct_parent['Segment'], child = 'Segment')
    posItr += seg_vsod_count
    ind +=1
    
else:
    ind+=1

In [ ]:
print(ind,posItr)

42 101


In [ ]:
if len(subsegments)>0:
    newVolumeSold(prs, subseg_vsod_merged, position=posItr, parent=direct_parent['SubSegment'], child = 'SubSegment')
    posItr += subseg_vsod_count
    ind+=1
else:
    ind+=1

In [ ]:
print(ind,posItr)

43 101


In [ ]:
if len(subcategories)>0:
    newVolumeSold(prs, subcat_vsod_merged, position=posItr, parent=direct_parent['SubCategory'], child = 'SubCategory')
    posItr += subcat_vsod_count
    ind+=1
else:
    ind+=1

In [ ]:

print(ind,posItr)

44 101


In [ ]:
# slide 11
for key,value in Scope.items():
    dict = {key: count_df(modified_promotionBrandsP12M,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in modified_promotionBrandsP12M.items() if key in dict[key1]}
        if filtered_dict:
            PromoShare_vs_ValueShare(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

49 113


In [ ]:
# slide 12
for key,value in Scope.items():
    dict = {key: count_df(newModifiedBrands,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in newModifiedBrands.items() if key in dict[key1]}
        if filtered_dict:
            PromoSalesTotalSize(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

Manual Shave Men | Walmart
6
System | Walmart
6
Disposables | Walmart
6
Disposables | Walmart
6
Razors | Walmart
6
Refills | Walmart
6


In [ ]:
print(ind,posItr)

54 119


In [ ]:
# slide 13
if promo_type:
    for key,value in Scope.items():
        dict = {key: count_df(PromoSalesTypes_data,value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in PromoSalesTypes_data.items() if key in dict[key1]}
            if filtered_dict:
                PromoSalesTypes(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
            posItr += len(filtered_dict)
            ind +=1
else:
    ind +=5

Manual Shave Men | Bj's And Sam's
Manual Shave Men | Walmart
System | Bj's And Sam's
Disposables | Bj's And Sam's
System | Walmart
Disposables | Walmart
Disposables | Bj's And Sam's
Razors | Bj's And Sam's
Disposables | Walmart
Razors | Walmart
Refills | Walmart


In [ ]:
print(ind,posItr)

59 130


In [ ]:
# slide 14
if feature_share:
    for key,value in Scope.items():
        dict = {key: count_df(modified_promotionBrandsP12M,value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in modified_promotionBrandsP12M.items() if key in dict[key1]}
            if filtered_dict:    
                featureShare(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
            posItr += len(filtered_dict)
            ind +=1
else:
    ind +=5

In [ ]:

# slide 15
if display_share:
    for key,value in Scope.items():
        dict = {key: count_df(modified_promotionBrandsP12M,value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in modified_promotionBrandsP12M.items() if key in dict[key1]}
            if filtered_dict:    
                displayShare(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
            posItr += len(filtered_dict)
            ind +=1
else:
    ind +=5

In [ ]:
print(ind,posItr)

69 130


In [ ]:
# slide 16
for key,value in Scope.items():
    dict = {key: count_df(modified_promotionEndOfWeek,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in modified_promotionEndOfWeek.items() if key in dict[key1]}
        if filtered_dict:
            PromoFrequency(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

74 163


In [ ]:
if categories:
    catFinal = sorted(splitDfsPromo(dfCategory,(client_manuf) ,genrateIndexList(catGroup[0])[0]))
    catFinal = catFinal+sorted(splitDfsPromo(dfCategory,(client_brands) ,genrateIndexList(catGroup[0])[0]))
    catFinal

In [ ]:
print(ind,posItr)

74 163


In [ ]:
if sectors:
    secFinal = sorted(splitDfsPromo(dfSector,(client_manuf)  ,genrateIndexList(secGroup[0])[0]))
    secFinal = secFinal + sorted(splitDfsPromo(dfSector,(client_brands)  ,genrateIndexList(secGroup[0])[0]))
    secFinal

In [ ]:
print(ind,posItr)

74 163


In [ ]:
if segments:
    segFinal = sorted(splitDfsPromo(dfSegment,(client_manuf)  ,genrateIndexList(segGroup[0])[0]))
    segFinal = segFinal+sorted(splitDfsPromo(dfSegment,(client_brands)  ,genrateIndexList(segGroup[0])[0]))
    segFinal

In [ ]:
if subsegments:
    subsegFinal = sorted(splitDfsPromo(dfSubSegment,(client_manuf)  ,genrateIndexList(subsegGroup[0])[0]))
    subsegFinal = subsegFinal + sorted(splitDfsPromo(dfSubSegment,(client_brands)  ,genrateIndexList(subsegGroup[0])[0]))
    subsegFinal

In [ ]:
if subcategories:
    subcatFinal = sorted(splitDfsPromo(dfSubCategory,(client_manuf) ,genrateIndexList(subcatGroup[0])[0]))
    subcatFinal = subcatFinal+sorted(splitDfsPromo(dfSubCategory,(client_brands) ,genrateIndexList(subcatGroup[0])[0]))
    subcatFinal

In [ ]:
print(ind,posItr)

74 163


In [ ]:
catGroup

[["Manual Shave Men | Edgewell Personal Care | Bj's And Sam's",
  "Manual Shave Men | Schick | Bj's And Sam's",
  "Manual Shave Men | Cremo | Bj's And Sam's"],
 ['Manual Shave Men | Edgewell Personal Care | Walmart',
  'Manual Shave Men | Schick | Walmart',
  'Manual Shave Men | Equate | Walmart',
  'Manual Shave Men | Cremo | Walmart']]

In [ ]:
#Slide 17
#split catGroup into Lists depends on num of charts 
catGroupSplit = splitListpromo(dfCategory, catGroup, [i-14 for i in index[ind]])
print(catGroupSplit)
promoSalesPerRetailer(prs,dfCategory,len(index[ind]),catGroupSplit,position=sum(duplication[:ind]))
posItr = sum(duplication[:ind]) + len(index[ind])
ind+=1
print(len(index[ind]),ind, posItr)

#split secGroup into Lists depends on num of charts 
if len(sectors) != 0: 
    secGroupSplit = splitListpromo(dfSector, secGroup, [i-14 for i in index[ind]])
    promoSalesPerRetailer(prs,dfSector,len(index[ind]),secGroupSplit,position=posItr)
    posItr += len(index[ind])
ind+=1
print(len(index[ind]),ind, posItr)

#split segGroup into Lists depends on num of charts 
if len(segments) != 0: 
    segGroupSplit = splitListpromo(dfSegment, segGroup, [i-14 for i in index[ind]])
    promoSalesPerRetailer(prs,dfSegment,len(index[ind]),segGroupSplit,position=posItr)
    posItr += len(index[ind])
ind+=1
print(len(index[ind]),ind, posItr)

#split subsegGroup into Lists depends on num of charts 
if len(subsegments) != 0:
    subsegGroupSplit = splitListpromo(dfSubSegment, subsegGroup, [i-14 for i in index[ind]])

    promoSalesPerRetailer(prs,dfSubSegment,len(index[ind]),subsegGroupSplit,position=posItr)
    posItr += len(index[ind])
ind+=1
print(len(index[ind]),ind, posItr)

#split subcatGroup into Lists depends on num of charts 
if len(subcategories) != 0:
    subcatGroupSplit = splitListpromo(dfSubCategory, subcatGroup, [i-14 for i in index[ind]])
    print(subcatGroupSplit)
    promoSalesPerRetailer(prs,dfSubCategory,len(index[ind]),subcatGroupSplit,position=posItr)
    posItr += len(index[ind])
ind+=1


[["Manual Shave Men | Edgewell Personal Care | Bj's And Sam's", "Manual Shave Men | Schick | Bj's And Sam's", "Manual Shave Men | Cremo | Bj's And Sam's"], ['Manual Shave Men | Edgewell Personal Care | Walmart', 'Manual Shave Men | Schick | Walmart', 'Manual Shave Men | Equate | Walmart', 'Manual Shave Men | Cremo | Walmart']]
4 75 165
5 76 169
0 77 174
0 78 174


In [ ]:
# slide 21
for key,value in Scope.items():
    dict = {key: count_df(modified_valueUplift,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in modified_valueUplift.items() if key in dict[key1]}
        if filtered_dict:
            valueUplift(prs,filtered_dict,duplication[ind],position=posItr)
        posItr += len(filtered_dict)
        print(ind,posItr)
        ind +=1


79 179
80 186
81 196
82 196
83 196


In [ ]:
print(ind, duplication[ind], posItr)

84 1 196


In [ ]:
if len(categories)>0:
    seasonality(prs,dfCategory0, len(index[ind]), categoryGroup0, position=posItr,slideby="Category")
    posItr += len(index[ind])
ind+=1
print(posItr, len(index[ind])) 

Edgewell Personal Care
Edgewell Personal Care
198 2


In [ ]:
if len(sectors)>0:
    seasonality(prs, dfSector0, len(index[ind]), secGroup0, position=posItr,slideby="Sector")
    posItr += len(index[ind])
ind+=1
print(posItr, len(index[ind])) 

Manual Shave Men
Manual Shave Men
200 4


In [ ]:
if len(segments)>0:
    seasonality(prs, dfSegment0, len(index[ind]), segGroup0, position=posItr,slideby="Segment")
    posItr += len(index[ind])
ind+=1
print(posItr, len(index[ind]),ind)  

Disposables
Disposables
System
System
204 0 87


In [ ]:
if len(subsegments) != 0:
    seasonality(prs,dfSubSegment0,len(index[ind]),subsegGroup0,position=posItr,slideby="SubSegment")
    posItr += len(index[ind])
ind+=1

print(posItr, len(index[ind]),ind)  

204 0 88


In [ ]:
if len(subcategories) != 0:
    seasonality(prs,dfSubCategory0,len(index[ind]),subcatGroup0,position=posItr,slideby="SubCategory")
    posItr += len(index[ind])
ind+=1


In [ ]:
print(len(index[ind]), ind, duplication[ind], posItr)

2 89 1 204


In [ ]:
print(posItr,ind)

204 89


In [ ]:
end_date1 = pd.to_datetime(end_date)
start_date1 = end_date1 - pd.DateOffset(months=12)
 
# Generate all weekly periods (weekly ends, e.g., Sundays) between start and end
week_ends = pd.date_range(start=start_date1, end=end_date1, freq='W-SUN')
 
# Convert to list of dates (if needed)
week_ends_list = week_ends.to_list()
all_weeks_df = pd.DataFrame({'End of Week': week_ends_list})

def add_all_weeks(data):
    final_data ={}
    for key,df in data.items():
        df_full = all_weeks_df.merge(df, on='End of Week', how='left')
        df_full.fillna(0, inplace=True)
        final_data[key] = df_full
    return final_data

In [ ]:
modified_dfCategory1 = add_all_weeks(modified_dfCategory1)
modified_dfSector1 = add_all_weeks(modified_dfSector1)
modified_dfSegment1 = add_all_weeks(modified_dfSegment1)

In [142]:
catGroup1Split = splitListpromo(modified_dfCategory1, catGroup1, [i-25 for i in index[ind]])

Promotional_Frequency(prs,modified_dfCategory1,len(index[ind]),catGroup1Split,position=posItr)
posItr +=len(catGroup1Split)
ind+=1
print(len(index[ind]), posItr)
#Sector Replace
if len(sectors) != 0: 
    secGroup1Split = splitListpromo(modified_dfSector1, secGroup1, [i-25 for i in index[ind]])
    Promotional_Frequency(prs,modified_dfSector1,len(index[ind]),secGroup1Split,position=posItr)
    posItr += len(secGroup1Split)
ind+=1
print(len(index[ind]), posItr)


if len(segments) != 0: 
    segGroup1Split = splitListpromo(modified_dfSegment1, segGroup1, [i-25 for i in index[ind]])
    Promotional_Frequency(prs,modified_dfSegment1,len(index[ind]),segGroup1Split,position=posItr)
    posItr += len(segGroup1Split)
ind+=1
print(len(index[ind]), posItr)

if len(subsegments) != 0:
    subsegGroup1Split = splitListpromo(modified_dfSubSegment1, subsegGroup1, [i-25 for i in index[ind]])
    Promotional_Frequency(prs,modified_dfSubSegment1,len(index[ind]),subsegGroup1Split,position=posItr)
    posItr += len(subsegGroup1Split)
ind+=1
print(len(index[ind]), posItr)

if len(subcategories) != 0:
    subcatGroup1Split = splitListpromo(modified_dfSubCategory1, subcatGroup1, [i-25 for i in index[ind]])
    Promotional_Frequency(prs,modified_dfSubCategory1,len(index[ind]),subcatGroup1Split,position=posItr)
    posItr += len(subcatGroup1Split)
ind+=1


0
1
2
0
1
2
4 206
0
1
0
1
2
0
1
0
1
2
3
5 210
0
1
0
1
2
0
1
0
1
2
0
1
2
3
0 215
0 215


In [143]:
# #slide 1 with no client brands
# for key,value in Scope.items():
#     dict = {key: count_df(modified_promotionBrandsP12M,value) }
#     for key1,value1 in dict.items():
#         filtered_dict = {key: value for key, value in modified_promotionBrandsP12M.items() if key in dict[key1]}
#         if filtered_dict:
#             promoValueSales_no(prs,filtered_dict,duplication[ind],position=posItr)
#             posItr += len(filtered_dict)
#         ind+=1

In [144]:
print(posItr)

215


In [145]:
# slide 2 with no client brands
for key,value in Scope.items():
    dict = {key: count_df(promotionsBrandNOTSortedTotalFinal,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in promotionsBrandNOTSortedTotalFinal.items() if key in dict[key1]}
        if filtered_dict:
            promoEvolutionNew(prs,filtered_dict,duplication[ind],position=posItr)
            posItr += len(filtered_dict)
        ind +=1


In [146]:
print(posItr)

227


In [147]:
# slide 3 with no client brands
for key,value in Scope.items():
    dict = {key: count_df(newpromotionsNotBrandsWithMarket,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in newpromotionsNotBrandsWithMarket.items() if key in dict[key1]}
        if filtered_dict:
            VSOD1(prs,filtered_dict,duplication[ind],position=posItr)
        posItr += len(filtered_dict)
        ind +=1

In [148]:
print(posItr)

231


In [149]:
# slide 4 with no client brands
for key,value in Scope.items():
    dict = {key: count_df(concated,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in concated.items() if key in dict[key1]}
        if filtered_dict:
            valueUpliftRetailer_no(prs,filtered_dict,duplication[ind],position=posItr)
        posItr += len(filtered_dict)
        ind +=1

In [150]:
print(posItr)

237


In [151]:
# slide 11 with no client prio
for key,value in Scope.items():
    dict = {key: count_df(modified_promotionBrandsP12M,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in modified_promotionBrandsP12M.items() if key in dict[key1]}
        if filtered_dict:
            PromoShare_vs_ValueShare_no(prs,filtered_dict,duplication[ind],position=posItr)
        posItr += len(filtered_dict)
        ind +=1

In [152]:
print(posItr)

249


In [153]:
print(ind, posItr)


114 249


In [154]:
# slide 12 with no client prio
for key,value in Scope.items():
    dict = {key: count_df(newModifiedBrands,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in newModifiedBrands.items() if key in dict[key1]}
        if filtered_dict:
            PromoSalesTotalSize_no(prs,filtered_dict,duplication[ind],position=posItr)
        posItr += len(filtered_dict)
    ind +=1
    print(posItr,ind)  
        

6
7
250 115
6
7
6
9
252 116
6
9
6
9
6
8
255 117
255 118
255 119


In [155]:
# slide 13 with no client prio
# if promo_type:
#     for key,value in Scope.items():
#         dict = {key: count_df(PromoSalesTypes_data,value) }
#         for key1,value1 in dict.items():
#             filtered_dict = {key: value for key, value in PromoSalesTypes_data.items() if key in dict[key1]}
#             if filtered_dict:
#                 PromoSalesTypes_no(prs,filtered_dict,duplication[ind],position=posItr)
#             posItr += len(filtered_dict)
#             ind +=1
# else:
#     ind +=5

In [156]:
# slide 14 with no client prio
if feature_share:
    for key,value in Scope.items():
        dict = {key: count_df(modified_promotionBrandsP12M,value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in modified_promotionBrandsP12M.items() if key in dict[key1]}
            if filtered_dict:    
                featureShare_no(prs,filtered_dict,duplication[ind],position=posItr)
            posItr += len(filtered_dict)
            ind +=1
else:
    ind +=5
    

In [157]:
# slide 15 with no client prio
if display_share:
    for key,value in Scope.items():
        dict = {key: count_df(modified_promotionBrandsP12M,value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in modified_promotionBrandsP12M.items() if key in dict[key1]}
            if filtered_dict:    
                displayShare_no(prs,filtered_dict,duplication[ind],position=posItr)
            posItr += len(filtered_dict)
            ind +=1
else:
    ind +=5

## Output slides

In [158]:
outputPath=os.getcwd() + f"\\Promotion {client_manuf[0]}.pptx"
prs.save(outputPath)
# # app = win32.Dispatch("PowerPoint.Application")
final=os.getcwd() +f"\\Promotion {client_manuf[0]}.pptx"
open_chart_data_in_excel(final,outputPath)

Slide 17: Opened Excel workbook: Book1
Slide 18: Opened Excel workbook: Book1
Slide 19: Opened Excel workbook: Book1
Slide 20: Opened Excel workbook: Book1
Slide 21: Opened Excel workbook: Book1
Slide 22: Opened Excel workbook: Book1
Slide 23: Opened Excel workbook: Book1
Slide 24: Opened Excel workbook: Book1
Slide 25: Opened Excel workbook: Book1
Slide 26: Opened Excel workbook: Book1
Slide 27: Opened Excel workbook: Book1
Slide 28: Opened Excel workbook: Book1
Slide 29: Opened Excel workbook: Book1
Slide 30: Opened Excel workbook: Book1
Slide 31: Opened Excel workbook: Book1
Slide 32: Opened Excel workbook: Book1
Slide 33: Opened Excel workbook: Book1
Slide 34: Opened Excel workbook: Book1
Slide 35: Opened Excel workbook: Book1
Slide 36: Opened Excel workbook: Book1
Slide 37: Opened Excel workbook: Book1
Slide 38: Opened Excel workbook: Book1
Slide 39: Opened Excel workbook: Book1
Slide 40: Opened Excel workbook: Book1
Slide 41: Opened Excel workbook: Book1
Slide 42: Opened Excel wo

In [159]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\Promotion Slide Duplicate\Promotion Replacement Function.ipynb"

path1 = r"/ValueUpliftvsDepth/"
loaded_data = {}
datasets_path = os.getcwd()+ path1
datasets = os.listdir(datasets_path)
for d in datasets:
    with open(datasets_path+d, 'rb') as handle:
        loaded_data[d.split('.')[0]] = pd.read_csv(handle)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'c:\\Users\\aleaa\\Documents\\Slide-Automate\\Promotion Slide Duplicate/ValueUpliftvsDepth/'

In [ ]:
valueUplift_dict = {} # value
i=0
for key, df in loaded_data.items():
    data = DetectHeader(df)
    columns_to_ffill = [col for col in data.columns if 'item' in col.lower() or 'product' in col.lower()]
    data[columns_to_ffill] = data[columns_to_ffill].fillna(method='ffill')
    data = data[~data['Item'].str.contains('Total', case=False)].reset_index(drop=True)
    for item in data['Item'].unique():
        df = data[data['Item'] == item]
        df['Discount Depth (%)'] = df['Discount Depth (%)'].str.replace('%','').astype(float) /100
        df['Promo Price/Unit'] = df['Promo Price/Unit'].str.replace('£','').astype(float)
        if normalized:
            df['Value Uplift (v. base) Normalized'] = df['Value Uplift (v. base) Normalized'].str.replace('%','').astype(float) /100
        else:
            df['Value Uplift (v. base)'] = df['Value Uplift (v. base)'].str.replace().str.replace('%','').astype(float) /100
        df = df[df['End of Week'] != '0']
        df['End of Week'] = pd.to_datetime(df['End of Week'])
        df = df[(df['End of Week'] >= start_date) & (df['End of Week'] <= end_date)].reset_index(drop=True)
        if df.shape[0]>0 and not df['Discount Depth (%)'].isna().all():
            df = df.fillna(0).reset_index(drop = True)
            new_key = key+'_'+ item
            valueUplift_dict[new_key] = df


In [ ]:
sign = 'before'
decimals = 2
currency = '£'
data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending July 2024"


index = [20]
duplication = [len(valueUplift_dict.keys())]
section_names = ["Value Uplift by product"]
path = os.getcwd() + '//Promotion base Oct 2024.pptx'
new_pre = os.getcwd() + '//slide duplicated value.pptx'


In [ ]:
#slideDuplication(index,duplication,section_names,path,new_pre)

In [ ]:
prs = Presentation(new_pre)

In [ ]:
# Iterate through each key-slide_num pair in modified_valueUplift
for key, slide_num in zip(valueUplift_dict, range(len(valueUplift_dict.keys()))):
        # Access the slide to be modified
        slide = prs.slides[slide_num]
        
        # Extract data for the current key
        df = valueUplift_dict[key]
        #df = df[df['Value Uplift (v. base) Normalized'] !=0 ]
        # Get shapes in the slide
        shapes = slide.shapes
        
        # Find and update title shape
        titleNumber = get_shape_number(shapes, "Value Uplift vs discount depth | By Event | Category/Sector | Brand | Coop Alleanza | P12M")
        datasourcenum = get_shape_number(shapes, "Data Source | Trade Panel")
        headerNumber = get_shape_number(shapes, 'Value Uplift vs discount depth (Replace With SO WHAT)')
        if titleNumber is not None:
            shapes[datasourcenum].text = data_source
            shapes[titleNumber].text = shapes[titleNumber].text.replace('Category/Sector', key.split('_')[2]) \
                .replace('Brand | Coop Alleanza ', df['Item'][0])
            shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
            shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
            shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)
            shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'

        # Create table and chart objects
        tables, charts = createTableAndChart(slide.shapes)
        chart1 = charts[0].chart  # First chart
        chart2 = charts[1].chart  # Second chart
        
        # Extract data for charts
        category = df['Item'].tolist()
        x_values_discount = df['Discount Depth (%)'].tolist()
        x_values_price = df['Promo Price/Unit'].tolist()
        if normalized:
            y_values = df['Value Uplift (v. base) Normalized'].tolist()
        else:
            y_values = df['Value Uplift (v. base)'].tolist()

        
        x_values_discount = [mround_numpy(value, 0.05) for value in x_values_discount]
        x_values_price = [mround_numpy(value, 0.5) for value in x_values_price]
        #Update first chart with Discount Depth vs Value Uplift data
        chart_data1 = XyChartData()
        series1 = chart_data1.add_series('Scatter')
        for i in range(len(category)):
            series1.add_data_point(x_values_discount[i], y_values[i])
        chart1.replace_data(chart_data1)
        
        # Access the X-axis
        
        xlsx_file = BytesIO()
        with chart_data1._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
            chart_data1._workbook_writer._populate_worksheet(workbook, worksheet)
            worksheet.write(0, 4, "Item")
            worksheet.write_column(1, 4, df['Item'].to_list(), None)
            worksheet.write(0, 5, "End of Week")
            worksheet.write_column(1, 5, df['End of Week'].to_list(), None)

        chart1._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

        # Update second chart with Promo Price/Unit vs Value Uplift data
        chart_data2 = XyChartData()
        series2 = chart_data2.add_series('Scatter')
        for i in range(len(category)):
            series2.add_data_point(x_values_price[i], y_values[i])
        chart2.replace_data(chart_data2)
        
        x_axis = chart2.category_axis
        
        # Loop through each X-axis category label and format as currency
        if sign.lower() == 'before':
            x_axis.tick_labels.number_format = f'"{currency}"#,##0.00'  if decimals == 2 else f'"{currency}"#,##0'
        else:
            x_axis.tick_labels.number_format = f'#,##0.00"{currency}"'  if decimals == 2 else f'#,##0"{currency}"'
       
        #x_axis.has_major_gridlines = False  # Optional: remove gridlines

        xlsx_file = BytesIO()
        with chart_data2._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
            chart_data2._workbook_writer._populate_worksheet(workbook, worksheet)
            worksheet.write(0, 4, "Item")
            worksheet.write_column(1, 4, df['Item'].to_list(), None)
            worksheet.write(0, 5, "End of Week")
            worksheet.write_column(1, 5, df['End of Week'].to_list(), None)
        chart2._workbook.update_from_xlsx_blob(xlsx_file.getvalue())


In [ ]:
outputPath=os.getcwd() + "\\Promotion EdgeWell ValueUplift.pptx"
prs.save(outputPath)
app = win32.Dispatch("PowerPoint.Application")
presentation = app.Presentations.Open(outputPath)